# Set-up

## Imports

In [ ]:
import corr_utils
import corr_utils.covariate as utils
import corr_utils.analysis as analysis_utils
import corr_utils.ml as ml_utils
import corr_utils.extraction as pipeline

In [ ]:
import pandas as pd
import numpy as np
import operator

In [3]:
from ydata_profiling import ProfileReport
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from importlib import reload

reload(utils)
reload(pipeline)
reload(ml_utils)
reload(analysis_utils)

## Configurations

In [7]:
corr_utils.set_default_key(key='case_id') # default key for merging

In [8]:
conn, error = pipeline.connect_impala(
    remote_hostname='hdl-edge01.charite.de', 
    username='nokr10'
    ) # connect to HDL

In [9]:
limit = None # amount of rows to load (for faster exploration)

In [10]:
extraction_date = "CAST( `_hdl_loadstamp` AS DATE) <= '2024-09-05'" # set for reproducibility

In [11]:
train_test_split = 0.2 # split used for training and evaluation

In [12]:
# pipeline.disconnect_impala(conn) # disconnect from HDL

# Raw Data

## Hierarchy

In [13]:
where = (
    '(c_var_name = "Behandlung_OP_Zeiten_Ereignisname" AND c_value IN ("BEGAN", "FREIG", "ENDEL", "SCHNI", "EINSC", "ENDAN", "NAHT", "BEGAW")) OR ' # OP times
    'c_var_name IN ("BEH_ANAE_ASA_STATUS", "Risiko_ASA", "Behandlung_Anae_Praemed_ASA_Status", "Praemedikation_ASA_Status") OR ' # ASA status
    'c_var_name IN ("Patient_Gewicht", "Behandlung_Gewicht", "Behandlung_Gewicht_Aufnahme", "CO_klinStatus_Behandlung_Patient_Aufnahme_Gewicht_", "CO_Patient_Aufnahme_Gewicht") OR ' # weight
    'c_var_name IN ("Patient_Groesse", "Praemedikation_Groesse", "CO_klinStatus_Behandlung_Patient_Aufnahme_Groesse_", "CO_Patient_Aufnahme_Groesse")' # height
)

df_hdl_copra_hierarchy = pipeline.get_impala_df(
    database='db_corror_prepared', 
    table='it_copra6_hierarchy_v2', 
    conn=conn, 
    limit=limit, 
    where=where + ' AND ' + extraction_date
    )

In [14]:
df_hierarchy = utils.extract_df_data(
    df_hdl_copra_hierarchy, 
    col_dict={
        'c_falnr':'case_id', 
        'c_var_name':'variable', 
        'c_value':'value', 
        'c_var_timestamp':'date_time'
        },
    remove_prefix=False,
    drop=True
    )

In [15]:
# convert data types

df_hierarchy = df_hierarchy.astype({
    # 'case_id': str,
    # 'variable': str,
    # 'value': ...,
})

for column in ['date_time']:
    df_hierarchy[f'{column}'] = pd.to_datetime(df_hierarchy[f'{column}'])

In [ ]:
df_hierarchy

## Prodecures (nicp)

In [17]:
df_hdl_sap_procedure = pipeline.get_impala_df(
    database='db_corror_prepared', 
    table='it_ishmed_procedure', 
    conn=conn, 
    limit=limit, 
    where=extraction_date
    )

In [18]:
df_nicp = utils.extract_df_data(
    df_hdl_sap_procedure, 
    col_dict={
        'c_falnr':'case_id', 
        'c_prozedur_code':'ops_code', 
        'c_prozedur_begin':'procedure_date_time'
        },
    remove_prefix=False,
    drop=True
    )

In [19]:
# convert data types

df_nicp = df_nicp.astype({
    # 'case_id': str,
    # 'ops_code': str,
})

for column in ['procedure_date_time']:
    df_nicp[f'{column}'] = pd.to_datetime(df_nicp[f'{column}'])

In [ ]:
df_nicp

## Patient (npat)

In [21]:
df_hdl_sap_patient = pipeline.get_impala_df(
    database='db_corror_prepared', 
    table='it_ishmed_patient', 
    conn=conn, 
    limit=limit, 
    where=extraction_date
    )

In [22]:
df_npat = utils.extract_df_data(
    df_hdl_sap_patient, 
    col_dict={
        'c_falnr':'case_id', 
        'c_gender':'gender', 
        'c_birthdate':'birth_date', 
        'c_datetimeofdeath':'death_date_time'
        },
    remove_prefix=False,
    drop=True
    )

In [ ]:
# check for duplicates
df_npat = utils.handle_duplicates(
    df=df_npat, 
    column='case_id', 
    drop_duplicates=True
    )

In [ ]:
df_npat

In [25]:
# convert data types (for direct processing)
df_npat['birth_date'] = pd.to_datetime(df_npat['birth_date'])
df_npat['death_date_time'] = pd.to_datetime(df_npat['death_date_time'], errors='coerce')

In [26]:
# add death indicator
df_npat['died'] = 0
df_npat.loc[df_npat['death_date_time'].notna(), 'died'] = 1
df_npat['died'] = df_npat['died'].astype(int)

In [ ]:
# remove unknowns (defined as 'Unknown')
df_npat = utils.exclude_rows(df=df_npat, column='gender', items=['Unknown'])

In [28]:
# adjust format
df_npat['female_sex'] = 0
df_npat['female_sex'] = (df_npat['gender'] == 'F').astype(int)

In [29]:
# remove old column
df_npat = df_npat.drop(columns=['gender'])

In [30]:
# convert data types

df_npat = df_npat.astype({
    # 'case_id': str,
    'female_sex': int,
    'died': int
})

for column in ['birth_date', 'death_date_time']:
    df_npat[f'{column}'] = pd.to_datetime(df_npat[f'{column}'])

In [ ]:
df_npat

## Lab

In [32]:
where = (
    'c_katalog_leistungtext LIKE "%Kreatinin%" AND c_wert <> "0"'
)

df_sap_lab = pipeline.get_impala_df(
    database='db_corror_prepared', 
    table='it_ishmed_labor', 
    conn=conn, 
    limit=limit, 
    where=where + ' AND ' + extraction_date
    )

In [33]:
df_lab = utils.extract_df_data(
    df=df_sap_lab, 
    col_dict={
        'c_falnr':'case_id', 
        'c_katalog_leistungtext':'substance', 
        'c_wert':'quantity', 
        'c_wert_einheit':'unit', 
        'c_wert_timestamp':'substance_date_time'
        },
    remove_prefix=False,
    drop=True
    )

In [34]:
# convert to numeric
df_lab['quantity'] = pd.to_numeric(df_lab['quantity'], errors='coerce')

In [ ]:
# remove non-numerics
df_lab_cleaned = df_lab.dropna(subset=['quantity']) 
utils.get_amount_removed_rows(
    initial=df_lab, 
    new=df_lab_cleaned
    )
df_lab = df_lab_cleaned

In [36]:
# convert data types

df_lab = df_lab.astype({
    # 'case_id': str,
    # 'substance': str,
    'quantity': float,
    # 'unit': str,
})

for column in ['substance_date_time']:
    df_lab[f'{column}'] = pd.to_datetime(df_lab[f'{column}'])

In [ ]:
df_lab

## Cases (cohort complete)

In [38]:
df_hdl_sap_fall = pipeline.get_impala_df(
    database='db_corror_prepared', 
    table='it_ishmed_fall', 
    conn=conn, 
    limit=limit, 
    where=extraction_date
    )

In [39]:
df_cohort_complete = utils.extract_df_data(
    df=df_hdl_sap_fall, 
    col_dict={
        'c_patnr':'pat_id',
        'c_falnr':'case_id', 
        'c_aufnahme':'admission_date_time', 
        'c_entlassung':'discharge_date_time'
        }, 
    remove_prefix=False,
    drop=True)

In [40]:
# convert data types

df_cohort_complete = df_cohort_complete.astype({
    # 'case_id': str,
    # 'pat_id': str
})

for column in ['admission_date_time', 'discharge_date_time']:
    df_cohort_complete[f'{column}'] = pd.to_datetime(df_cohort_complete[f'{column}'])

In [ ]:
df_cohort_complete

## Diagnoses (ndia)

In [50]:
df_hdl_copra_diagnose = pipeline.get_impala_df(
    database='db_corror_prepared', 
    table='it_ishmed_diagnose', 
    conn=conn, 
    limit=limit, 
    where=extraction_date
    )

In [43]:
df_ndia = utils.extract_df_data(
    df=df_hdl_copra_diagnose, 
    col_dict={
        'c_falnr':'case_id', 
        'c_diagnose_1':'icd_code', 
        'c_diagnose_timestamp':'diagnosis_date_time', 
        'c_gewissheit':'certainty'
        }, 
    remove_prefix=False,
    drop=True
    )

In [ ]:
# exclude "Ausgeschlossen" or "Verworfen" (https://health-data.charite.de/data-model/ish-med/ndia/diagw)
df_ndia = utils.exclude_rows(
    df=df_ndia, 
    column='certainty', 
    items=['A', 'VW'], 
    drop=True
    )

In [551]:
# convert data types

df_ndia = df_ndia.astype({
    # 'case_id': str,
    # 'icd_code': str
})

for column in ['diagnosis_date_time']:
    df_ndia[f'{column}'] = pd.to_datetime(df_ndia[f'{column}'])

In [ ]:
df_ndia

## Medication

In [553]:
where = (
    "LOWER(c_generic_name) LIKE '%insulin%'" # insulin
)

df_hdl_copra_medication = pipeline.get_impala_df(
    database='db_corror_prepared', 
    table='it_copra6_medication', 
    conn=conn, 
    limit=limit, 
    where=where + ' AND ' + extraction_date
    )

In [554]:
df_medication = utils.extract_df_data(
    df=df_hdl_copra_medication, 
    col_dict={
        'c_falnr':'case_id', 
        'c_generic_name':'medication', 
        'c_application_start':'medication_date_time'
        }, 
    remove_prefix=False,
    drop=True
    )

In [555]:
# convert to datetime
df_medication['medication_date_time'] = pd.to_datetime(df_medication['medication_date_time'], utc=True) # UTC+1 to UTC
df_medication['medication_date_time'] = df_medication['medication_date_time'].dt.strftime('%Y-%m-%d %H:%M:%S') # remove timezone information

In [556]:
# convert data types

df_medication = df_medication.astype({
    # 'case_id': str,
    # 'medication': str,
})

for column in ['medication_date_time']:
    df_medication[f'{column}'] = pd.to_datetime(df_medication[f'{column}'])

In [ ]:
df_medication

## Movement

In [558]:
where = (
    'c_bewegungsart = "OP" AND ' # surgeries
    '(c_pflege_oe_id LIKE "S%" OR c_pflege_oe_id LIKE "W%" OR c_pflege_oe_id LIKE "M%")' # information about campus
    )

df_sap_movement = pipeline.get_impala_df(
    database='db_corror_prepared', 
    table='it_ishmed_bewegung', 
    conn=conn, 
    limit=limit, 
    where=where + ' AND ' + extraction_date
    )

In [559]:
df_campi = utils.extract_df_data(
    df=df_sap_movement, 
    col_dict={'c_falnr':'case_id', 'c_pflege_oe_id':'station', 'c_begin':'movement_start_date_time', 'c_ende':'movement_end_date_time'}, 
    remove_prefix=False,
    drop=True
    )

In [560]:
# get campus information (first letter)
df_campi['campus'] = df_campi['station'].str[0]

In [561]:
# sort by dates (to ensure earliest are at the top)
df_campi = df_campi.sort_values(by='movement_start_date_time', ascending=True)

In [ ]:
# keep only first OP (for each case)
df_campi = utils.handle_duplicates(
    df=df_campi, 
    column='case_id',
    drop_duplicates=True
    )

In [ ]:
# remove edge cases
for column in ['movement_start_date_time', 'movement_end_date_time']:
    df_campi_cleaned = df_campi[
        (df_campi[column] >= pd.Timestamp.min) &
        (df_campi[column] <= pd.Timestamp.max)
    ]
utils.get_amount_removed_rows(
    initial=df_campi, 
    new=df_campi_cleaned
    )
df_campi = df_campi_cleaned

In [564]:
# convert data types

df_campi = df_campi.astype({
    # 'case_id': str,
    # 'station': str
})

for column in ['movement_start_date_time', 'movement_end_date_time']:
    df_campi[f'{column}'] = pd.to_datetime(df_campi[f'{column}'])

In [ ]:
df_campi

# Cohort

In [566]:
# all cases with OP
df_op = utils.extract_df_data(
    df=df_nicp, 
    col_dict={
        'procedure_date_time':'op_date_time'
        }, 
    filter_dict={
        'ops_code':['^5']
        },
    drop=False
    )

In [ ]:
df_op

In [568]:
# convert data types

df_op = df_op.astype({
    # 'case_id': str,
    # 'ops_code': str,
})

for column in ['op_date_time']:
    df_op[f'{column}'] = pd.to_datetime(df_op[f'{column}'])

In [ ]:
df_op

In [ ]:
# remove missing dates
df_op = utils.exclude_rows(
    df=df_op, 
    column='op_date_time', 
    items=[pd.NaT]
    )

In [ ]:
df_op

In [572]:
# sort by dates (to ensure earliest are at the top)
df_cohort_complete = df_cohort_complete.sort_values(by='admission_date_time', ascending=True)
df_op = df_op.sort_values(by='op_date_time')

In [ ]:
# keep only first OP (for each case)
df_op = utils.handle_duplicates(
    df=df_op, 
    column='case_id', 
    drop_duplicates=True
    )

In [ ]:
df_op

In [ ]:
# only keep cohort cases with OP
df_op_cohort = pd.merge(df_cohort_complete, df_op, on='case_id', how='inner')
utils.get_amount_removed_rows(
    initial=df_cohort_complete, 
    new=df_op_cohort
    )

In [ ]:
df_op_cohort

In [ ]:
# keep only first OP for each patient (among cases)
df_cohort = utils.handle_duplicates(
    df=df_op_cohort, 
    column='pat_id', 
    drop_duplicates=True
    )

In [ ]:
df_cohort

In [ ]:
# remove missing dates
df_cohort = utils.exclude_rows(
    df=df_cohort, 
    column='admission_date_time', 
    items=[pd.NaT]
    )

In [ ]:
# remove missing dates
df_cohort = utils.exclude_rows(
    df=df_cohort, 
    column='discharge_date_time', 
    items=[pd.NaT]
    )

In [ ]:
df_cohort

In [ ]:
# remove cases that are outside of stay

conditions = [
    (lambda row: (row['admission_date_time'] < row['op_date_time']) and (row['op_date_time'] < row['discharge_date_time']), 'OP_between_stay')
]

df_cohort_cleaned = utils.create_subgroups(
    df=df_cohort, 
    conditions=conditions
    )

df_cohort_cleaned_removed = utils.exclude_rows(
    df=df_cohort_cleaned, 
    column='OP_between_stay', 
    items=[0]
    )
df_cohort_cleaned_removed.drop(columns=['OP_between_stay'], inplace=True)
df_cohort = df_cohort_cleaned_removed

In [ ]:
df_cohort

In [584]:
# add campi
df_cohort = pd.merge(
    df_cohort, df_campi[['case_id', 'campus', 'movement_start_date_time', 'movement_end_date_time']], 
    on='case_id', how='left'
    )

In [ ]:
df_cohort

In [ ]:
# remove cases with op outside movements

conditions = [
    (lambda row: (row['movement_start_date_time'] <= row['op_date_time']) and (row['op_date_time'] <= row['movement_end_date_time']), 'movement_between_stay')
]

df_cohort_cleaned = utils.create_subgroups(
    df=df_cohort, 
    conditions=conditions
    )

df_cohort_cleaned_removed = utils.exclude_rows(
    df=df_cohort_cleaned, 
    column='movement_between_stay', 
    items=[0]
    )
df_cohort_cleaned_removed = df_cohort_cleaned
df_cohort_cleaned_removed.drop(columns=['movement_between_stay', 'movement_start_date_time', 'movement_end_date_time'], inplace=True)
df_cohort = df_cohort_cleaned_removed

In [ ]:
df_cohort

In [589]:
# convert data types

df_cohort = df_cohort.astype({
    # 'case_id': str,
    # 'pat_id': str,
    # 'ops_code': str,
    # 'campus': str
})

for column in ['admission_date_time', 'discharge_date_time', 'op_date_time']:
    df_cohort[f'{column}'] = pd.to_datetime(df_cohort[f'{column}'])

In [ ]:
df_cohort

# Derived Data

#### Prior Diagnoses

In [ ]:
# only keep ICDs before OP
df_prior_diagnoses = utils.filter_time_between(
    df=df_ndia, time_column='diagnosis_date_time', 
    df_time_reference=df_cohort, 
    df_time_reference_column_upper_bound='op_date_time', 
    merge_on='case_id', 
    drop=True
    )

In [592]:
# convert data types

df_prior_diagnoses = df_prior_diagnoses.astype({
    # 'case_id': str,
    # 'icd_code': str
})

for column in ['diagnosis_date_time']:
    df_prior_diagnoses[f'{column}'] = pd.to_datetime(df_prior_diagnoses[f'{column}'])

In [ ]:
df_prior_diagnoses

#### Prior Medication

In [ ]:
# only keep medications before OP
df_prior_medication = utils.filter_time_between(
    df=df_medication, time_column='medication_date_time', 
    df_time_reference=df_cohort, 
    between=True, 
    df_time_reference_column_upper_bound='op_date_time', 
    df_time_reference_column_lower_bound='admission_date_time',
    merge_on='case_id', 
    drop=True
    )

In [595]:
# convert data types

df_prior_medication = df_prior_medication.astype({
    # 'case_id': str,
    # 'medication': str,
})

for column in ['medication_date_time']:
    df_prior_medication[f'{column}'] = pd.to_datetime(df_prior_medication[f'{column}'])

In [ ]:
df_prior_medication

#### Prior Lab

In [ ]:
# only keep labs before OP
df_prior_lab = utils.filter_time_between(
    df=df_lab, 
    time_column='substance_date_time', 
    df_time_reference=df_cohort, 
    between=True, 
    df_time_reference_column_upper_bound='op_date_time', 
    df_time_reference_column_lower_bound='admission_date_time',
    merge_on='case_id', 
    drop=True
    )

In [598]:
# convert data types

df_prior_lab = df_prior_lab.astype({
    # 'case_id': str,
    # 'substance': str,
    'quantity': float,
    # 'unit': str,
})

for column in ['substance_date_time']:
    df_prior_lab[f'{column}'] = pd.to_datetime(df_prior_lab[f'{column}'])

In [ ]:
df_prior_lab

#### 30 days after OP

In [600]:
# get 30 days after OP
df_cohort_temp = df_cohort[['case_id', 'op_date_time']].copy()
df_cohort_temp['30_days'] = df_cohort['op_date_time'] + pd.Timedelta(days=30)

#### Diagnoses 30 days

In [ ]:
# only keep ICDs 30 days after OP
df_diagnoses_30_days = utils.filter_time_between(
    df=df_ndia, 
    time_column='diagnosis_date_time', 
    df_time_reference=df_cohort_temp, 
    between=True,
    df_time_reference_column_upper_bound='30_days', 
    df_time_reference_column_lower_bound='op_date_time', 
    merge_on='case_id', 
    drop=True
    )

In [602]:
# convert data types

df_diagnoses_30_days = df_diagnoses_30_days.astype({
    # 'case_id': str,
    # 'icd_code': str
})

for column in ['diagnosis_date_time']:
    df_diagnoses_30_days[f'{column}'] = pd.to_datetime(df_diagnoses_30_days[f'{column}'])

In [ ]:
df_diagnoses_30_days

#### Procedures 30 days

In [ ]:
# only keep OPS 30 days after OP
df_procedures_30_days = utils.filter_time_between(
    df=df_nicp, 
    time_column='procedure_date_time', 
    df_time_reference=df_cohort_temp, 
    between=True, 
    df_time_reference_column_upper_bound='30_days', 
    df_time_reference_column_lower_bound='op_date_time', 
    merge_on='case_id', 
    drop=True
    )

In [605]:
# convert data types

df_procedures_30_days = df_procedures_30_days.astype({
    # 'case_id': str,
    # 'ops_code': str
})

for column in ['procedure_date_time']:
    df_procedures_30_days[f'{column}'] = pd.to_datetime(df_procedures_30_days[f'{column}'])

In [ ]:
df_procedures_30_days

# Variables

## Variables (Basic)

In [607]:
# add given variables to cohort
df_cohort = pd.merge(
    df_cohort, df_npat[['case_id', 'female_sex', 'birth_date']],
    on='case_id', how='left'
    )

In [608]:
# calculate (and add) age at time of OP
df_cohort['age_during_op'] = df_cohort['op_date_time'].dt.year - df_cohort['birth_date'].dt.year

In [609]:
# make NaN = -1
df_cohort['age_during_op'] = df_cohort['age_during_op'].fillna(-1)

In [ ]:
# remove cases with missing values
df_cohort = utils.exclude_rows(df_cohort, 'age_during_op', [-1])

In [611]:
# convert data types

df_cohort = df_cohort.astype({
    # 'case_id': str,
    # 'pat_id': str,
    # 'ops_code': str,
    'female_sex': int,
    'age_during_op': int
})

for column in ['admission_date_time', 'discharge_date_time', 'op_date_time', 'birth_date']:
    df_cohort[f'{column}'] = pd.to_datetime(df_cohort[f'{column}'])

In [ ]:
df_cohort

## Variables (Complex)

### OP Times [WIP]

In [613]:
# extract by priority

df_op_start_time = utils.extract_by_priority(
    df=df_hierarchy, 
    column='value', 
    priority_order=['BEGAN', 'FREIG', 'ENDEL', 'SCHNI', 'EINSC']
    )

df_op_end_time = utils.extract_by_priority(
    df=df_hierarchy, 
    column='value', 
    priority_order=['ENDAN', 'NAHT', 'BEGAW']
    )

In [ ]:
df_op_start_time = utils.handle_duplicates(
    df=df_op_start_time, 
    column='case_id', 
    drop_duplicates=True
    )

In [ ]:
df_op_end_time = utils.handle_duplicates(
    df=df_op_end_time, 
    column='case_id', 
    drop_duplicates=True
    )

In [616]:
# rename columns
df_op_start_time.rename(columns={'date_time': 'op_start_date_time'}, inplace=True)
df_op_end_time.rename(columns={'date_time': 'op_end_date_time'}, inplace=True)

In [617]:
# merge with cohort
df_cohort = pd.merge(df_cohort, df_op_start_time[['case_id', 'op_start_date_time']], on='case_id', how='left')
df_cohort = pd.merge(df_cohort, df_op_end_time[['case_id', 'op_end_date_time']], on='case_id', how='left')

In [ ]:
df_cohort['op_start_date_time'].notna().sum()

In [ ]:
df_cohort['op_end_date_time'].notna().sum()

In [620]:
# TODO: filter for OP dates (once more data is available)

### OP Length

In [621]:
df_op_length = df_cohort[['case_id', 'op_start_date_time', 'op_end_date_time']].copy()

In [622]:
# calculate length
df_op_length['op_length'] = (df_op_length['op_end_date_time'] - df_op_length['op_start_date_time']).dt.total_seconds() / 60.0

In [623]:
# convert data types
df_op_length = df_op_length.astype({
    # 'case_id': str,
    'op_length': float
})

In [624]:
# merge with cohort
df_cohort = pd.merge(df_cohort, df_op_length[['case_id', 'op_length']], on='case_id', how='left')

In [ ]:
df_cohort['op_length'].notna().sum()

### ASA Status [WIP]

In [626]:
# extract by priority
df_asa_status = utils.extract_by_priority(
    df=df_hierarchy, 
    column='variable', 
    priority_order=[
        'BEH_ANAE_ASA_STATUS', 'Risiko_ASA', 'Behandlung_Anae_Praemed_ASA_Status', 
        'Praemedikation_ASA_Status'
        ]
    )

In [ ]:
df_asa_status = utils.handle_duplicates(
    df=df_asa_status, 
    column='case_id', 
    drop_duplicates=True
    )

In [628]:
# rename columns
df_asa_status.rename(columns={'value': 'asa_status'}, inplace=True)

In [ ]:
# check variables
df_hierarchy['variable'].value_counts()

In [ ]:
# check unique value counts
df_asa_status['asa_status'].value_counts()

In [630]:
# extract all numbers

def extract_number(value):
    number = ''.join(filter(str.isdigit, str(value)))
    return int(number) if number else np.nan

df_asa_status['asa_status'] = df_asa_status['asa_status'].apply(extract_number)

In [ ]:
# check (new) unique value counts
df_asa_status['asa_status'].value_counts()

In [ ]:
df_asa_status_cleaned = df_asa_status.dropna(subset=['asa_status'])
utils.get_amount_removed_rows(
    initial=df_asa_status, 
    new=df_asa_status_cleaned
    )
df_asa_status = df_asa_status_cleaned

In [633]:
# convert data types

df_asa_status = df_asa_status.astype({
    # 'case_id': str,
    'asa_status': int
})

for column in ['date_time']:
    df_asa_status[f'{column}'] = pd.to_datetime(df_asa_status[f'{column}'])

In [634]:
# merge with cohort
df_cohort = pd.merge(df_cohort, df_asa_status[['case_id', 'asa_status']], on='case_id', how='left')

In [ ]:
df_cohort['asa_status'].notna().sum()

In [636]:
# TODO: filter for OP dates (once more data is available)

### Weight [WIP]

In [637]:
# extract by priority
df_weight = utils.extract_by_priority(
    df=df_hierarchy, 
    column='variable', 
    priority_order=[
        'Patient_Gewicht', 'Praemedikation_Gewicht', 'Behandlung_Gewicht', 'Behandlung_Gewicht_Aufnahme', 
        'CO_klinStatus_Behandlung_Patient_Aufnahme_Gewicht_', 'CO_Patient_Aufnahme_Gewicht'
        ]
    )

In [ ]:
df_weight = utils.handle_duplicates(
    df=df_weight, 
    column='case_id', 
    drop_duplicates=True
    )

In [639]:
# rename columns
df_weight.rename(columns={'value': 'weight'}, inplace=True)

In [640]:
# convert data types

df_weight = df_weight.astype({
    # 'case_id': str,
    'weight': float
})

for column in ['date_time']:
    df_weight[f'{column}'] = pd.to_datetime(df_weight[f'{column}'])

In [641]:
# merge with cohort
df_cohort = pd.merge(df_cohort, df_weight[['case_id', 'weight']], on='case_id', how='left')

In [ ]:
df_cohort['weight'].notna().sum()

In [643]:
# TODO: filter for OP dates (once more data is available)

### Height [WIP]

In [644]:
# extract by priority
df_height = utils.extract_by_priority(
    df=df_hierarchy, 
    column='variable', 
    priority_order=[
        'Patient_Groesse', 'Praemedikation_Groesse', 'CO_klinStatus_Behandlung_Patient_Aufnahme_Groesse_', 
        'CO_Patient_Aufnahme_Groesse'
        ]
    )

In [ ]:
df_height = utils.handle_duplicates(
    df=df_height, 
    column='case_id', 
    drop_duplicates=True
    )

In [646]:
# rename columns
df_height.rename(columns={'value': 'height'}, inplace=True)

In [647]:
# convert data types

df_height = df_height.astype({
    # 'case_id': str,
    'height': float
})

for column in ['date_time']:
    df_height[f'{column}'] = pd.to_datetime(df_height[f'{column}'])

In [648]:
# merge with cohort
df_cohort = pd.merge(df_cohort, df_height[['case_id', 'height']], on='case_id', how='left')

In [ ]:
df_cohort['height'].notna().sum()

In [650]:
# TODO: filter for OP dates (once more data is available)

### BMI

In [651]:
df_bmi = df_cohort[['case_id', 'weight', 'height']].copy()

In [652]:
df_bmi['bmi'] = df_bmi['weight'] / ((df_bmi['height'] * 0.01) ** 2)

In [653]:
# convert data types

df_bmi = df_bmi.astype({
    # 'case_id': str,
    'bmi': float
})

In [654]:
# merge with cohort
df_cohort = pd.merge(df_cohort, df_bmi[['case_id', 'bmi']], on='case_id', how='left')

In [ ]:
df_cohort['bmi'].notna().sum()

### Elixhauser Categories

#### Mapping

In [656]:
# get elixhauser definitions
df_elixhauser_definition = pd.read_csv('data/elixhauser.csv') # references: 10.1097/MLR.0b013e31819432e5; 10.1097/01.mlr.0000182534.19832.83

In [657]:
# map ICD-10 codes to Elixhauser categories (based on the implementation by Moritz Thiele)
icd_to_category = {}
for index, row in df_elixhauser_definition.iterrows():
    for code in row['icd_codes'].split('|'):
        icd_to_category[code] = row['category']

In [658]:
def map_icd_to_category(icd_code):
    icd_code_str = str(icd_code) 
    for code in icd_to_category:
        if icd_code_str.startswith(code): # add wildcard
            return icd_to_category[code]
    return None

In [659]:
df_prior_diagnoses_elixhauser = df_prior_diagnoses[['case_id', 'icd_code']].copy()

In [660]:
# map each diagnosis to its Elixhauser category
df_prior_diagnoses_elixhauser['elixhauser_category'] = df_prior_diagnoses_elixhauser['icd_code'].apply(map_icd_to_category)

In [661]:
df_prior_diagnoses_elixhauser.drop(columns=['icd_code'], inplace=True)

In [662]:
# pivot table to create presence indicators for each category 
df_presence_absence = pd.pivot_table(df_prior_diagnoses_elixhauser, index='case_id', columns='elixhauser_category', 
                          aggfunc=lambda x: 1 if len(x) > 0 else 0, fill_value=0)

In [663]:
# convert data types
# df_presence_absence['case_id'] = df_presence_absence['case_id'].astype(str)
category_columns = df_presence_absence.columns
df_presence_absence[category_columns] = df_presence_absence[category_columns].astype(int)

In [ ]:
df_presence_absence

#### Merge

In [665]:
df_presence_absence_cleaned = df_presence_absence.copy()

In [666]:
# clean up column names
df_presence_absence_cleaned.columns = df_presence_absence_cleaned.columns.str.lower().str.replace(' ', '_').str.replace(',', '') + '_elixhauser'

In [667]:
# add elixhauser categories to cohort
df_cohort = pd.merge(df_cohort, df_presence_absence_cleaned, on='case_id', how='left', suffixes=('', ''))

#### Missings

In [668]:
elixhauser_columns = list(df_presence_absence_cleaned.columns)

In [669]:
df_cohort[elixhauser_columns] = df_cohort[elixhauser_columns].fillna(0)

### Elevated-Risk Surgery

In [670]:
# get all cases with elevated risk surgery
df_elevated_risk_surgery = utils.extract_df_data(
    df=df_cohort, 
    filter_dict={
        'ops_code':[
            '^5-54', '^5-55', '^5-58', '^5-51', 
            '^5-32', '^5-35', '^5-42', '^5-38'
            ]
        }
    ) 

In [671]:
# add indicator to each case
df_cohort['elevated_risk_surgery'] = df_cohort['case_id'].isin(df_elevated_risk_surgery['case_id']).astype(int)

### Ischemic Heart Disease History (MI)

In [672]:
# get all cases with prior ischemic heart disease diagnosis
df_MI_history = utils.extract_df_data(
    df=df_prior_diagnoses, 
    filter_dict={
        'icd_code':['^I21', '^I22', 'I25.2']
        }
    ) # reference: https://doi.org/10.1016/j.cjco.2022.07.008

In [673]:
# add indicator to each case
df_cohort['MI_history'] = df_cohort['case_id'].isin(df_MI_history['case_id']).astype(int)

### Cerebrovascular Disease History (CD)

In [674]:
# get all cases with prior cerebrovascular disease diagnosis
df_CD_history = utils.extract_df_data(
    df=df_prior_diagnoses, 
    filter_dict={
        'icd_code':[
            '^G45', '^G46', 'H34.0', '^160', '^161', '^162', '^163', 
            '^164', '^165', '^166', '^167', '^168', '^169'
            ]
        }
    ) # references: https://doi.org/10.1016/j.cjco.2022.07.008; https://doi.org/10.1038/s41591-024-03206-0

In [675]:
# add indicator to each case
df_cohort['CD_history'] = df_cohort['case_id'].isin(df_CD_history['case_id']).astype(int)

### Prior Insulin

In [676]:
# get all cases with prior Insulin medication
df_insulin = utils.extract_df_data(
    df=df_prior_medication, 
    filter_dict={'medication': ['insulin']}
    )

In [677]:
# add indicator to each case
df_cohort['prior_insulin'] = df_cohort['case_id'].isin(df_insulin['case_id']).astype(int)

### Prior Creatinine

In [ ]:
df_prior_lab['substance'].value_counts()

In [679]:
# get all cases with prior creatinine (from blood)
df_creatinine = utils.extract_df_data(
    df=df_prior_lab, 
    filter_dict={
        'substance':[
            'Kreatinin (Jaffé) HP', 'Kreatinin (Jaffé)', 
            'Kreatinin (Jaffé) (HP)', 'Kreatinin (enzym.) HP'
            ]
    },
    exact_match=True
    ) 

In [ ]:
# check units
df_creatinine['unit'].value_counts()

In [ ]:
# remove all other units
df_creatinine = utils.exclude_rows(
    df=df_creatinine, 
    column='unit', 
    items=['mg/dl'], 
    filter_operator=operator.eq
    )

In [682]:
# convert (use if other units are used that should be kept)
conversion_factors = {
    'mg/dl': 1,
    'g/l': 1000
}

In [683]:
def convert_to_mg_dl(row):
    quantity = row['quantity']
    unit = row['unit']
    factor = conversion_factors.get(unit, 1)
    return quantity * factor

In [684]:
# convert quantities
df_creatinine['quantity_mg_dl'] = df_creatinine.apply(convert_to_mg_dl, axis=1)
df_creatinine = df_creatinine.drop(columns=['unit', 'quantity'])

In [685]:
# sum all creatinine
df_creatinine = utils.aggregate_data(
    df=df_creatinine, 
    column='quantity_mg_dl', 
    method=utils.AggregationMethod.SUM, 
    rename=True
    )

In [ ]:
# sufficient quantity
df_creatinine_cleaned = df_creatinine[df_creatinine['quantity_mg_dl_sum'] >= 2]
utils.get_amount_removed_rows(
    initial=df_creatinine, 
    new=df_creatinine_cleaned
    )
df_creatinine = df_creatinine_cleaned

In [ ]:
df_creatinine

In [688]:
# add indicator to each case
df_cohort['prior_creatinine'] = df_cohort['case_id'].isin(df_creatinine['case_id']).astype(int)

### Vascular Disease History 

In [689]:
# ICD codes for vascular disease 

vascular_disorder = [
    'I70.449',
    'I70.638',
    'I70.361',
    'I70.599',
    'I70.639',
    'I70.692',
    'I70.202',
    'I70.601',
    'E09.59',
    'I70.703',
    'I70.744',
    'I70.732',
    'I70.462',
    'I70.663',
    'I70.739',
    'I70.718',
    'I70.201',
    'I70.513',
    'E13.51',
    'I70.338',
    'I21.A9',
    'I43',
    'E10.65',
    'I70.398',
    'I70.369',
    'I42.7',
    'I70.411',
    'I70.748',
    'E10.59',
    'I70.661',
    'I70.221',
    'I70.721',
    'I73.1',
    'I70.341',
    'I70.743',
    'I70.518',
    'E11.51',
    'I70.461',
    'E10.51',
    'I70.529',
    'I70.745',
    'I21.3',
    'I70.245',
    'I70.331',
    'I70.218',
    'I70.618',
    'I22.9',
    'I70.499',
    'I70.301',
    'I42.0',
    'I70.432',
    'I70.269',
    'I22.0',
    'I70.345',
    'I70.441',
    'I70.442',
    'I70.428',
    'I42.6',
    'I70.528',
    'I21.09',
    'I70.568',
    'I70.702',
    'I70.708',
    'I70.212',
    'I70.791',
    'I70.335',
    'I21.02',
    'I70.535',
    'E08.59',
    'I42.5',
    'I70.348',
    'I70.409',
    'I70.738',
    'I21.9',
    'I70.291',
    'I70.544',
    'I21.01',
    'I70.608',
    'I70.769',
    'I73.9',
    'I70.213',
    'I70.238',
    'I70.332',
    'I70.362',
    'I70.445',
    'I70.593',
    'I70.649',
    'I70.422',
    'I70.719',
    'I70.723',
    'I70.733',
    'I70.512',
    'I70.313',
    'I70.55',
    'I70.693',
    'I70.292',
    'I70.648',
    'I70.645',
    'I70.519',
    'I70.641',
    'I70.368',
    'I70.643',
    'I70.399',
    'I70.448',
    'I70.35',
    'I70.799',
    'I70.344',
    'I70.633',
    'I70.642',
    'I70.239',
    'I70.25',
    'I70.491',
    'I70.592',
    'I42.2',
    'I70.268',
    'I70.534',
    'I70.731',
    'E11.65',
    'E13.52',
    'I70.603',
    'I70.219',
    'I22.1',
    'I70.363',
    'I42.8',
    'I70.349',
    'I70.492',
    'I70.562',
    'I70.735',
    'I70.91',
    'I70.435',
    'I70.208',
    'I70.45',
    'I70.634',
    'I70.662',
    'I70.793',
    'I70.545',
    'I70.669',
    'I70.629',
    'I70.561',
    'I70.768',
    'I70.298',
    'I70.328',
    'I70.429',
    'I70.232',
    'E11.59',
    'I70.699',
    'I70.8',
    'I70.542',
    'I70.498',
    'I70.1',
    'I70.309',
    'I70.209',
    'I70.244',
    'I70.713',
    'A18.84',
    'I70.318',
    'I70.401',
    'I70.631',
    'I70.749',
    'I70.728',
    'I70.668',
    'I70.233',
    'I70.623',
    'I70.763',
    'I70.333',
    'I70.611',
    'I70.722',
    'I70.509',
    'I70.533',
    'I70.563',
    'I70.229',
    'I70.393',
    'I70.299',
    'E08.52',
    'I42.1',
    'I70.235',
    'I70.241',
    'I70.262',
    'I70.311',
    'I42.4',
    'I70.391',
    'I70.402',
    'I70.522',
    'I70.303',
    'I70.691',
    'I22.2',
    'I70.334',
    'I70.439',
    'I70.742',
    'I70.792',
    'I70.263',
    'E13.59',
    'I70.632',
    'I21.A1',
    'I70.308',
    'I70.469',
    'E09.52',
    'I70.223',
    'I70.242',
    'I70.408',
    'I70.203',
    'I70.493',
    'I70.548',
    'I70.75',
    'I42.9',
    'I70.501',
    'I70.321',
    'I70.443',
    'I70.421',
    'E10.52',
    'I70.711',
    'I70.549',
    'I70.302',
    'I70.423',
    'I70.434',
    'I70.231',
    'I70.591',
    'I70.628',
    'I70.612',
    'I70.644',
    'I70.701',
    'I70.438',
    'I70.538',
    'I70.234',
    'I70.248',
    'I70.433',
    'I70.569',
    'I70.709',
    'I70.403',
    'E11.52',
    'I70.90',
    'I70.621',
    'I21.4',
    'I70.243',
    'I70.712',
    'I70.602',
    'I70.539',
    'I70.329',
    'I70.762',
    'I70.322',
    'E08.65',
    'I70.312',
    'I70.502',
    'I70.503',
    'I70.211',
    'I70.622',
    'I70.419',
    'I70.228',
    'I70.463',
    'I70.511',
    'I70.521',
    'I70.65',
    'I70.532',
    'I79.8',
    'I70.468',
    'I70.543',
    'I70.343',
    'I70.392',
    'I70.508',
    'I70.418',
    'I70.609',
    'I70.698',
    'I70.249',
    'I70.531',
    'I70.598',
    'I70.339',
    'I70.734',
    'I70.413',
    'I70.613',
    'I79.1',
    'I21.21',
    'I22.8',
    'I70.541',
    'I70.222',
    'I42.3',
    'I21.11',
    'I70.619',
    'I70.412',
    'I70.635',
    'I70.92',
    'E09.51',
    'I21.29',
    'I70.523',
    'I70.342',
    'I70.444',
    'E08.51',
    'I70.323',
    'I70.729',
    'I70.798',
    'I70.0',
    'I70.319',
    'I21.19',
    'I70.761',
    'I70.261',
    'I70.293',
    'I70.741',
    'I70.431'
    ] # reference: https://doi.org/10.1002/pds.4973

In [691]:
# get all cases with prior vascular disease diagnosis
df_vascular_history = utils.extract_df_data(
    df=df_prior_diagnoses, 
    filter_dict={'icd_code':vascular_disorder}
    )

In [692]:
# add indicator to each case
df_cohort['vascular_disease_history'] = df_cohort['case_id'].isin(df_vascular_history['case_id']).astype(int)

### Stroke / TIA / Thromboembolism History (STT)

In [693]:
STT = [
    'I63.423',
    'I74.11',
    'I63.239',
    'I60.12',
    'I74.2',
    'I63.119',
    'T80.0XXA',
    'I63.449',
    'I60.2',
    'I63.032',
    'I63.232',
    'I61.1',
    'I63.211',
    'I63.22',
    'I74.19',
    'I60.9',
    'I63.012',
    'I63.539',
    'I74.4',
    'I63.30',
    'I63.531',
    'I63.549',
    'I74.09',
    'I63.00',
    'I63.421',
    'I63.519',
    'I74.8',
    'I63.412',
    'I60.52',
    'I63.321',
    'I63.132',
    'G46.0',
    'I63.59',
    'I63.419',
    'I63.429',
    'I63.441',
    'I61.3',
    'I63.543',
    'G45.2',
    'I63.432',
    'I63.113',
    'I63.09',
    'G45.0',
    'I63.131',
    'T82.818A',
    'I74.9',
    'I61.2',
    'I60.4',
    'I63.339',
    'I63.219',
    'I63.031',
    'I67.89',
    'I60.10',
    'I26.02',
    'I63.20',
    'I63.411',
    'I61.0',
    'I74.5',
    'I63.139',
    'I63.111',
    'I63.331',
    'I63.533',
    'I63.323',
    'I63.40',
    'I63.532',
    'I60.7',
    'I63.422',
    'I60.30',
    'I60.00',
    'I60.51',
    'I61.9',
    'T82.817A',
    'I63.349',
    'I63.311',
    'I63.319',
    'G45.8',
    'I63.212',
    'I63.513',
    'I26.90',
    'I74.3',
    'I63.233',
    'I63.343',
    'I63.333',
    'I63.512',
    'I63.8',
    'I74.10',
    'I63.133',
    'I60.50',
    'I63.341',
    'I63.542',
    'G45.1',
    'I61.6',
    'I60.8',
    'G45.9',
    'I63.439',
    'I63.541',
    'I63.39',
    'I63.50',
    'I63.413',
    'I63.6',
    'I63.431',
    'I67.848',
    'I63.013',
    'I63.19',
    'I26.01',
    'I61.4',
    'I60.6',
    'T81.718A',
    'I63.313',
    'I63.529',
    'I63.29',
    'I63.231',
    'I26.09',
    'I60.11',
    'I61.8',
    'I60.32',
    'I63.12',
    'I63.521',
    'I63.442',
    'I60.02',
    'I63.342',
    'I60.31',
    'I26.92',
    'I63.523',
    'I63.329',
    'I63.033',
    'I60.01',
    'I67.841',
    'I74.01',
    'I61.5',
    'I63.443',
    'I63.332',
    'I63.10',
    'G46.2',
    'I63.322',
    'G46.1',
    'I63.02',
    'I63.312',
    'I63.511',
    'I63.011',
    'I63.522',
    'I63.433',
    'I26.99',
    'I63.019',
    'I63.49',
    'I63.9',
    'T81.72XA',
    'I63.213',
    'I63.112',
    'I63.039'
 ] # reference: https://doi.org/10.1002/pds.4973

In [694]:
# get all cases with prior STT diagnosis
df_STT_history = utils.extract_df_data(
    df=df_prior_diagnoses, 
    filter_dict={'icd_code':STT}
    ) 

In [695]:
# add indicator to each case
df_cohort['STT_history'] = df_cohort['case_id'].isin(df_STT_history['case_id']).astype(int)

### Atrial Fibrillation (AF)

In [696]:
# get all cases with prior atrial fibrillation and flutter diagnosis
df_AF_history = utils.extract_df_data(
    df=df_prior_diagnoses, 
    filter_dict={'icd_code':['^I48']}
    ) # reference: https://doi.org/10.1038/s41591-024-03206-0

In [697]:
# add indicator to each case
df_cohort['AF_history'] = df_cohort['case_id'].isin(df_AF_history['case_id']).astype(int)

### Expanded RCRI Features

In [ ]:
df_expanded_RCRI_weights = pd.read_csv('data/expanded-RCRI-logit-weights.csv')
df_expanded_RCRI_weights

In [699]:
# clean features
filtered_values = df_expanded_RCRI_weights['feature'][df_expanded_RCRI_weights['feature'].str.endswith('_ICD_history')]
cleaned_values = filtered_values.str.replace('_ICD_history', '', regex=False)
expanded_RCRI_features = cleaned_values.tolist()

In [700]:
for feature in expanded_RCRI_features:    
    icd_pattern = f'^{feature}'
    df_feature = utils.extract_df_data(
        df=df_prior_diagnoses, 
        filter_dict={'icd_code':[icd_pattern]}
        )
    df_cohort[f'{feature}_ICD_history'] = df_cohort['case_id'].isin(df_feature['case_id']).astype(int)

### Expanded CHA2DS2-VASc Features

In [ ]:
df_expanded_CHA_weights = pd.read_csv('data/expanded-CHA-logit-weights.csv')
df_expanded_CHA_weights

In [702]:
# clean features
filtered_values = df_expanded_CHA_weights['feature'][df_expanded_CHA_weights['feature'].str.endswith('_ICD_history')]
cleaned_values = filtered_values.str.replace('_ICD_history', '', regex=False)
expanded_CHA_features = cleaned_values.tolist()

In [703]:
for feature in expanded_CHA_features:    
    icd_pattern = f'^{feature}'
    df_feature = utils.extract_df_data(
        df=df_prior_diagnoses, 
        filter_dict={'icd_code':[icd_pattern]}
        )
    df_cohort[f'{feature}_ICD_history'] = df_cohort['case_id'].isin(df_feature['case_id']).astype(int)

### Expanded Elixhauser Features

In [ ]:
df_expanded_elix_weights = pd.read_csv('data/expanded-elixhauser-logit-weights.csv')
df_expanded_elix_weights

In [705]:
# clean features
filtered_values = df_expanded_elix_weights['feature'][df_expanded_elix_weights['feature'].str.endswith('_ICD_history')]
cleaned_values = filtered_values.str.replace('_ICD_history', '', regex=False)
expanded_elix_features = cleaned_values.tolist()

In [706]:
for feature in expanded_elix_features:    
    icd_pattern = f'^{feature}'
    df_feature = utils.extract_df_data(
        df=df_prior_diagnoses, 
        filter_dict={'icd_code':[icd_pattern]}
        )
    df_cohort[f'{feature}_ICD_history'] = df_cohort['case_id'].isin(df_feature['case_id']).astype(int)

## Outcomes

### Mortality

In [707]:
# get variables for cases that died
df_mortality = utils.extract_df_data(df_npat, filter_dict={'died': [1]})

In [ ]:
# keep only those during hospital stay of OP cases
df_mortality = utils.filter_time_between(
    df=df_mortality, 
    time_column='death_date_time', 
    df_time_reference=df_cohort, 
    between=True, 
    df_time_reference_column_upper_bound='discharge_date_time', 
    df_time_reference_column_lower_bound='admission_date_time', 
    merge_on='case_id'
    )

In [709]:
# add indicator to cohort
df_cohort['in_hospital_death'] = df_cohort['case_id'].isin(df_mortality['case_id']).astype(int)

### Stroke

In [710]:
stroke_forward_backward = [
    'I74.11',
    'I26.09',
    'I74.2',
    'I26.90',
    'I74.3',
    'T80.0XXA',
    'T82.818A',
    'I74.9',
    'I26.92',
    'I74.19',
    'I74.10',
    'I26.02',
    'I74.4',
    'I74.01',
    'I74.5',
    'I74.09',
    'I74.8',
    'I26.99',
    'I26.01',
    'T82.817A',
    'T81.72XA',
    'T81.718A'
    ] # reference: https://doi.org/10.1002/pds.4973

In [711]:
stroke_nature = ['^I63.', 'I67.81', 'I67.89', 'I67.9', 'G45.1', 'G45.8', 'I67.89'] # reference: https://doi.org/10.1038/s41591-024-03206-0

In [712]:
# codes for stroke 
OPS_stroke = ['^8-836.80'] 
ICD_stroke = stroke_forward_backward + stroke_nature 

#### Diagnoses

In [713]:
# get all cases with stroke diagnosis 30 days after OP
df_ICD_stroke = utils.extract_df_data(
    df=df_diagnoses_30_days, 
    filter_dict={'icd_code':ICD_stroke}
    )

In [714]:
df_ICD_stroke = df_ICD_stroke[['case_id', 'icd_code']]

In [715]:
# convert data types
df_ICD_stroke = df_ICD_stroke.astype({
    # 'case_id': str,
    # 'icd_code': str
})

In [ ]:
df_ICD_stroke

In [ ]:
df_ICD_stroke['icd_code'].value_counts().head(10)

#### Prodecures

In [718]:
# get all cases with stroke procedure
df_OPS_stroke = utils.extract_df_data(
    df=df_procedures_30_days, 
    filter_dict={'ops_code': OPS_stroke}
    )

In [719]:
df_OPS_stroke  = df_OPS_stroke[['case_id', 'ops_code']]

In [720]:
# convert data types
df_OPS_stroke = df_OPS_stroke.astype({
    # 'case_id': str,
    # 'ops_code': str
})

In [ ]:
df_OPS_stroke

In [ ]:
df_OPS_stroke['ops_code'].value_counts().head(10)

#### Combination

In [723]:
ICD_OPS_stroke = set(df_ICD_stroke['case_id']).union(set(df_OPS_stroke['case_id']))

In [724]:
# add to cohort
df_cohort['stroke_30_days'] = df_cohort['case_id'].isin(ICD_OPS_stroke).astype(int) 

### MACE

In [725]:
# codes for MACE 

OPS_MACE = ['^5-36', '^8-84'] 

acute_myocardial_infarction = ['^I21' '^I22'] # reference: https://doi.org/10.1016/j.cjco.2022.07.008
non_fatal_cardiac_arrest_or_ventricular_arrythmia = ['I47.2', 'I49.01', 'I49.02', 'I46.9', 'I49.9', '^R99'] # reference: https://doi.org/10.1016/j.cjco.2022.07.008
ICD_MACE = acute_myocardial_infarction + non_fatal_cardiac_arrest_or_ventricular_arrythmia

#### Diagnoses

In [726]:
# get all cases with MACE diagnosis 30 days after OP
df_ICD_MACE = utils.extract_df_data(
    df=df_diagnoses_30_days, 
    filter_dict={'icd_code':ICD_MACE}
    )

In [727]:
df_ICD_MACE  = df_ICD_MACE[['case_id', 'icd_code']]

In [728]:
# convert data types
df_ICD_MACE = df_ICD_MACE.astype({
    # 'case_id': str,
    # 'icd_code': str
})

In [ ]:
df_ICD_MACE

In [ ]:
df_ICD_MACE['icd_code'].value_counts().head(10)

#### Prodecures

In [731]:
# get all cases with MACE procedure 30 days after OP
df_OPS_MACE = utils.extract_df_data(
    df=df_procedures_30_days, 
    filter_dict={'ops_code':OPS_MACE}
    )

In [732]:
df_OPS_MACE  = df_OPS_MACE[['case_id', 'ops_code']]

In [733]:
# convert data types
df_OPS_MACE = df_OPS_MACE.astype({
    # 'case_id': str,
    # 'ops_code': str
})

In [ ]:
df_OPS_MACE

In [ ]:
df_OPS_MACE['ops_code'].value_counts().head(10)

#### Combination

In [736]:
ICD_OPS_MACE = set(df_ICD_MACE['case_id']).union(set(df_OPS_MACE['case_id']))

In [737]:
ICD_OPS_MACE_DEATH =  set(ICD_OPS_MACE).union(set(df_mortality['case_id'])) # reference: https://doi.org/10.1016/j.cjco.2022.07.008

In [738]:
df_cohort['MACE_30_days'] = df_cohort['case_id'].isin(ICD_OPS_MACE_DEATH).astype(int) # add to cohort

# Scores

## Elixhauser

### Original

based on the implementation by Moritz Thiele

In [739]:
# get weights
# ahrq_weights = df_elixhauser_definition.set_index('category')['AHRQ_elixhauser_weights'].to_dict()
van_walraven_weights = df_elixhauser_definition.set_index('category')['van_walraven_elixhauser_weights'].to_dict()

In [740]:
# make score mutually exclusive for variants of hypertension and diabetes
df_presence_absence.loc[df_presence_absence['Diabetes, complicated'] == 1, 'Diabetes, uncomplicated'] = 0
df_presence_absence.loc[df_presence_absence['Hypertension, complicated'] == 1, 'Hypertension, uncomplicated'] = 0

In [741]:
# calculate scores
van_walraven_scores = df_presence_absence.mul(van_walraven_weights, axis=1).sum(axis=1)
# ahrq_scores = df_presence_absence.mul(ahrq_weights, axis=1).sum(axis=1)

In [742]:
# combine scores
df_elixhauser = pd.DataFrame({
    'case_id': df_presence_absence.index,
    # 'elixhauser_AHRQ': ahrq_scores.values,
    'elixhauser_van_walraven': van_walraven_scores.values
})

In [743]:
# fill missing
df_elixhauser = df_elixhauser.fillna(0)

In [744]:
# convert data types
df_elixhauser = df_elixhauser.astype({
    # 'case_id': str,
    # 'icd_code': str
    # 'elixhauser_AHRQ': int,
    'elixhauser_van_walraven': int
})

In [745]:
# add to cohort
df_cohort = pd.merge(df_cohort, df_elixhauser, on='case_id', how='left')

In [746]:
# fill missings (of cohort)
elixhauser_columns = ['elixhauser_van_walraven']
df_cohort[elixhauser_columns] = df_cohort[elixhauser_columns].fillna(0)

### Recalibrated

In [747]:
df_elixhauser = df_cohort.copy()

In [ ]:
df_elixhauser_weights = pd.read_csv('data/elixhauser-logit-weights.csv')
df_elixhauser_weights

In [749]:
elixhauser_features = [
    'valvular_disease_elixhauser', 'blood_loss_anemia_elixhauser', 'alcohol_abuse_elixhauser', 
    'peptic_ulcer_disease_excluding_bleeding_elixhauser', 'psychoses_elixhauser', 'hypertension_uncomplicated_elixhauser', 
    'cardiac_arrhythmias_elixhauser', 'other_neurological_disorders_elixhauser', 'depression_elixhauser', 
    'hypertension_complicated_elixhauser', 'solid_tumor_without_metastasis_elixhauser', 'drug_abuse_elixhauser', 
    'diabetes_complicated_elixhauser', 'lymphoma_elixhauser', 'rheumatoid_arthritis/collagen_vascular_diseases_elixhauser', 
    'deficiency_anemia_elixhauser', 'metastatic_cancer_elixhauser', 'obesity_elixhauser', 'liver_disease_elixhauser', 
    'hypothyroidism_elixhauser', 'diabetes_uncomplicated_elixhauser', 'paralysis_elixhauser', 'aids/hiv_elixhauser', 
    'chronic_pulmonary_disease_elixhauser', 'pulmonary_circulation_disorders_elixhauser', 'peripheral_vascular_disorders_elixhauser', 
    'weight_loss_elixhauser', 'renal_failure_elixhauser', 'congestive_heart_failure_elixhauser', 'fluid_and_electrolyte_disorders_elixhauser', 
    'coagulopathy_elixhauser'
    ]

new_elixhauser_weights = analysis_utils.load_weights(
    df=df_elixhauser_weights, 
    feature_column=elixhauser_features
    )

In [750]:
df_elixhauser['elixhauser_recalibrated'] = sum(
    df_elixhauser[feature] * new_elixhauser_weights[feature] for feature in elixhauser_features
)

In [751]:
df_elixhauser = df_elixhauser[['case_id', 'elixhauser_recalibrated']]

In [752]:
# convert data types
df_elixhauser = df_elixhauser.astype({
    # 'case_id': str,
    'elixhauser_recalibrated': float,
})

In [ ]:
df_elixhauser

In [754]:
# add to cohort
df_cohort = pd.merge(df_cohort, df_elixhauser, on='case_id', how='left')

## Expanded Elixhauser

In [755]:
df_expanded_elix = df_cohort.copy()

In [ ]:
df_expanded_elix_weights = pd.read_csv('data/expanded-elixhauser-logit-weights.csv')
df_expanded_elix_weights

In [757]:
expanded_elix_features = list(df_expanded_elix_weights[df_expanded_elix_weights['feature'] != 'const']['feature'])

expanded_elix_weights = analysis_utils.load_weights(
    df=df_expanded_elix_weights, 
    feature_column=expanded_elix_features
    )

In [ ]:
df_expanded_elix['expanded_elixhauser'] = 0

In [759]:
df_expanded_elix['expanded_elixhauser'] = sum(
    df_expanded_elix[feature] * df_expanded_elix[feature] for feature in expanded_elix_features
)

In [760]:
df_expanded_elix = df_expanded_elix[['case_id', 'expanded_elixhauser']]

In [761]:
# convert data types
df_expanded_elix = df_expanded_elix.astype({
    # 'case_id': str,
    'expanded_elixhauser': float,
})

In [ ]:
df_expanded_elix

In [763]:
df_cohort = pd.merge(df_cohort, df_expanded_elix, on='case_id', how='left')

## Cardiac Risk (RCRI)

> based on [MDCalc](https://www.mdcalc.com/calc/1739/revised-cardiac-risk-index-pre-operative-risk)

- high-risk surgery  -> intraperitoneal, intrathoracic, suprainguinal vascula, as obove (based on OPS of op) (```sap_nicp.csv```)
- history of MI -> I21*, I22*, I23* (based on ICD-dx before OP) (```sap_ndia.csv```)
- heart failure -> CHF (congestive heart failure) from Elixhauser [as in ```elix.ipynb```]
- history of cerebrovascular disease  -> I63*, I65*, I66*, I67*, I68*, I69* (based on ICD-dx) (```sap_ndia.csv```)
- preoperative insulin -> insulin in medications of co6 (```co6_medication.csv```)
- preoperative creatinine >2 (last lab preop) -> from labs sap

In [764]:
df_RCRI = df_cohort.copy()

In [765]:
# original
df_RCRI['RCRI_original'] = df_RCRI['elevated_risk_surgery'] * 1 \
    + df_RCRI['MI_history'] * 1 \
    + df_RCRI['congestive_heart_failure_elixhauser'] * 1 \
    + df_RCRI['CD_history'] * 1 \
    + df_RCRI['prior_insulin'] * 1 \
    + df_RCRI['prior_creatinine'] * 1

In [ ]:
df_RCRI_weights = pd.read_csv('data/RCRI-logit-weights.csv')
df_RCRI_weights

In [767]:
RCRI_features = [
    'elevated_risk_surgery', 'MI_history', 'congestive_heart_failure_elixhauser', 
    'CD_history', 'prior_insulin', 'prior_creatinine'
    ]

new_RCRI_weights = analysis_utils.load_weights(
    df=df_RCRI_weights, 
    feature_column=RCRI_features
    )

In [768]:
# recalibrated
df_RCRI['RCRI_recalibrated'] = df_RCRI['elevated_risk_surgery'] * new_RCRI_weights['elevated_risk_surgery'] \
    + df_RCRI['MI_history'] * new_RCRI_weights['MI_history'] \
    + df_RCRI['congestive_heart_failure_elixhauser'] * new_RCRI_weights['congestive_heart_failure_elixhauser'] \
    + df_RCRI['CD_history'] * new_RCRI_weights['CD_history']  \
    + df_RCRI['prior_insulin'] * new_RCRI_weights['prior_insulin']  \
    + df_RCRI['prior_creatinine'] * new_RCRI_weights['prior_creatinine']

In [769]:
df_RCRI = df_RCRI[['case_id', 'RCRI_recalibrated', 'RCRI_original']]

In [770]:
# convert data types
df_RCRI = df_RCRI.astype({
    # 'case_id': str,
    'RCRI_recalibrated': float,
    'RCRI_original': float
})

In [ ]:
df_RCRI

In [772]:
# add to cohort
df_cohort = pd.merge(df_cohort, df_RCRI, on='case_id', how='left')

## Cardiac Risk (Expanded RCRI)

In [773]:
df_expanded_RCRI = df_cohort.copy()

In [ ]:
df_expanded_RCRI_weights = pd.read_csv('data/expanded-RCRI-logit-weights.csv')
df_expanded_RCRI_weights

In [775]:
expanded_RCRI_features = list(df_expanded_RCRI_weights[df_expanded_RCRI_weights['feature'] != 'const']['feature'])

expanded_RCRI_weights = analysis_utils.load_weights(
    df=df_expanded_RCRI_weights, 
    feature_column=expanded_RCRI_features
    )

In [776]:
df_expanded_RCRI['expanded_RCRI'] = sum(
    df_expanded_RCRI[feature] * expanded_RCRI_weights[feature] for feature in expanded_RCRI_features
)

In [777]:
df_expanded_RCRI = df_expanded_RCRI[['case_id', 'expanded_RCRI']]

In [778]:
# convert data types
df_expanded_RCRI = df_expanded_RCRI.astype({
    # 'case_id': str,
    'expanded_RCRI': float,
})

In [ ]:
df_expanded_RCRI

In [780]:
# add to cohort
df_cohort = pd.merge(df_cohort, df_expanded_RCRI, on='case_id', how='left')

## Stroke Risk (CHA2DS2-VASc)

> based on [MDCalc](https://www.mdcalc.com/calc/801/cha2ds2-vasc-score-atrial-fibrillation-stroke-risk)

In [781]:
df_CHA = df_cohort.copy()

In [782]:
# get risk (points) for each age category
def age_risk(age):
    if age < 65:
        return 0
    elif age >= 65 and age < 75:
        return 1
    else:
        return 2

In [783]:
# original
df_CHA['CHA2DS2_VASc_original'] = df_CHA['female_sex'] * 1 \
    + df_CHA['congestive_heart_failure_elixhauser'] * 1 \
    + (df_CHA['hypertension_uncomplicated_elixhauser'] + df_CHA['hypertension_complicated_elixhauser']) * 1 \
    + (df_CHA['diabetes_uncomplicated_elixhauser'] + df_CHA['diabetes_complicated_elixhauser']) * 1 \
    + df_CHA['vascular_disease_history'] * 1 \
    + df_CHA['STT_history'] * 2 \
    + df_CHA['age_during_op'].apply(age_risk)

In [ ]:
df_CHA_weights = pd.read_csv('data/CHA-logit-weights.csv')
df_CHA_weights

In [785]:
CHA_features = {
    'female_sex', 'congestive_heart_failure_elixhauser', 'hypertension_uncomplicated_elixhauser', 
    'hypertension_complicated_elixhauser', 'diabetes_uncomplicated_elixhauser', 'diabetes_complicated_elixhauser', 
    'vascular_disease_history', 'STT_history', 'age_below_65', 'age_between_65_and_74', 'age_above_74'
    }

new_CHA_weights = analysis_utils.load_weights(
    df=df_CHA_weights, 
    feature_column=CHA_features
    )

In [786]:
def age_risk_recalibrated(age):
    if age < 65:
        return new_CHA_weights['age_below_65']
    elif age >= 65 and age < 75:
        return new_CHA_weights['age_between_65_and_74']
    else:
        return new_CHA_weights['age_above_74']

In [787]:
# recalibrated
df_CHA['CHA2DS2_VASc_recalibrated'] = df_CHA['female_sex'] * new_CHA_weights['female_sex'] \
    + df_CHA['congestive_heart_failure_elixhauser'] * new_CHA_weights['congestive_heart_failure_elixhauser'] \
    + df_CHA['hypertension_uncomplicated_elixhauser'] * new_CHA_weights['hypertension_uncomplicated_elixhauser'] \
    + df_CHA['hypertension_complicated_elixhauser'] * new_CHA_weights['hypertension_complicated_elixhauser'] \
    + df_CHA['diabetes_uncomplicated_elixhauser'] * new_CHA_weights['diabetes_complicated_elixhauser'] \
    + df_CHA['diabetes_complicated_elixhauser'] * new_CHA_weights['diabetes_uncomplicated_elixhauser'] \
    + df_CHA['vascular_disease_history'] * new_CHA_weights['vascular_disease_history'] \
    + df_CHA['STT_history'] * new_CHA_weights['STT_history'] \
    + df_CHA['age_during_op'].apply(age_risk_recalibrated)

In [788]:
df_CHA = df_CHA[['case_id', 'CHA2DS2_VASc_recalibrated', 'CHA2DS2_VASc_original']]

In [789]:
# convert data types
df_CHA = df_CHA.astype({
    # 'case_id': str,
    'CHA2DS2_VASc_recalibrated': float,
    'CHA2DS2_VASc_original': float
})

In [ ]:
df_CHA

In [791]:
# add to cohort
df_cohort = pd.merge(df_cohort, df_CHA, on='case_id', how='left')

## Stroke Risk (Expanded CHA2DS2-VASc)

In [792]:
df_expanded_CHA = df_cohort.copy()

In [ ]:
df_expanded_CHA_weights = pd.read_csv('data/expanded-CHA-logit-weights.csv')
df_expanded_CHA_weights

In [794]:
expanded_CHA_features = list(df_expanded_CHA_weights[df_expanded_CHA_weights['feature'] != 'const']['feature'])

expanded_CHA_weights = analysis_utils.load_weights(
    df=df_expanded_CHA_weights, 
    feature_column=expanded_CHA_features
    )

In [795]:
def age_risk_expanded(age):
    if age < 65:
        return expanded_CHA_weights['age_below_65']
    elif age >= 65 and age < 75:
        return expanded_CHA_weights['age_between_65_and_74']
    else:
        return expanded_CHA_weights['age_above_74']

In [ ]:
df_expanded_CHA


In [ ]:
expanded_CHA_features

In [798]:
df_expanded_CHA['expanded_CHA2DS2_VASc'] = 0

for feature in expanded_CHA_features:
    if feature != 'age_below_65' and feature != 'age_between_65_and_74' and feature != 'age_above_74':
        df_expanded_CHA['expanded_CHA2DS2_VASc'] += df_expanded_CHA[feature] * expanded_CHA_weights[feature]

df_expanded_CHA['expanded_CHA2DS2_VASc'] += df_expanded_CHA['age_during_op'].apply(age_risk_expanded)

In [799]:
df_expanded_CHA = df_expanded_CHA[['case_id', 'expanded_CHA2DS2_VASc']]

In [800]:
# convert data types
df_expanded_CHA = df_expanded_CHA.astype({
    # 'case_id': str,
    'expanded_CHA2DS2_VASc': float,
})

In [ ]:
df_expanded_CHA

In [802]:
# add to cohort
df_cohort = pd.merge(df_cohort, df_expanded_CHA, on='case_id', how='left')

## Pulmonary Risk (ARISCAT) [WIP]

In [803]:
# TBI

# Clinically Implausible Values

In [ ]:
df_cohort = utils.clean_values(
    df=df_cohort, 
    reference_values='data/reference-values.csv', 
    drop_rows=False # only set to missing
    )

# Exclusion

In [ ]:
# remove all cardiac surgeries
df_cohort_cleaned = df_cohort[~df_cohort['ops_code'].str.startswith(('5-35', '5-36', '5-37'))] # reference: https://klassifikationen.bfarm.de/ops/kode-suche/htmlops2023/block-5-35...5-37.htm
utils.get_amount_removed_rows(
    initial=df_cohort, 
    new=df_cohort_cleaned
    )
df_cohort = df_cohort_cleaned

In [ ]:
df_cohort

In [807]:
df_cohort.reset_index(inplace=True, drop=True)

In [ ]:
df_cohort = utils.clean_values(
    df=df_cohort, 
    reference_values='data/reference-values-exclusion.csv', 
    drop_rows=True
    ) 

In [809]:
start_date = '2005-01-01' # as there is a big jump from 2004 to 2005
end_date = '2024-09-05'

In [ ]:
df_cohort = utils.exclude_rows(
    df=df_cohort, 
    column='admission_date_time', 
    items=[start_date], 
    filter_operator=operator.le
    )

In [ ]:
df_cohort = utils.exclude_rows(
    df=df_cohort, 
    column='discharge_date_time', 
    items=[start_date], 
    filter_operator=operator.le
    )

In [ ]:
df_cohort = utils.exclude_rows(
    df=df_cohort, 
    column='op_date_time', 
    items=[start_date], 
    filter_operator=operator.le
    )

In [ ]:
df_cohort = utils.exclude_rows(
    df=df_cohort, 
    column='admission_date_time', 
    items=[end_date], 
    filter_operator=operator.ge
    )

In [ ]:
df_cohort = utils.exclude_rows(
    df=df_cohort, 
    column='discharge_date_time', 
    items=[end_date], 
    filter_operator=operator.ge
    )

In [ ]:
df_cohort = utils.exclude_rows(
    df=df_cohort, 
    column='op_date_time', 
    items=[end_date], 
    filter_operator=operator.ge
    )

In [ ]:
df_cohort

In [817]:
df_cohort.reset_index(inplace=True, drop=True)

In [ ]:
df_cohort

# Subgroups

In [26]:
conditions = [
    (lambda row: row['female_sex'] == 1, 'female'),
    (lambda row: row['female_sex'] == 0, 'male'),
    (lambda row: row['campus'] == 'M', 'campus_mitte'),
    (lambda row: row['campus'] == 'S', 'campus_steglitz'),
    (lambda row: row['campus'] == 'W', 'campus_wedding'),
    (lambda row: row['age_during_op'] > 65, 'age_above_65'),
    (lambda row: row['age_during_op'] < 65, 'age_below_65'),
    (lambda row: (row['age_during_op'] >= 65) and (row['age_during_op'] < 74), 'age_between_65_and_74'),
    (lambda row: row['age_during_op'] > 74, 'age_above_74'),
    (lambda row: row['asa_status'] <= 2, 'asa_le_2'),
    (lambda row: row['asa_status'] > 2, 'asa_gt_2'),
    (lambda row: row['admission_date_time'].day == row['op_date_time'].day == row['discharge_date_time'].day, 'ambulatory'),
    (lambda row: row['admission_date_time'].day < row['op_date_time'].day < row['discharge_date_time'].day, 'inpatient'),
    (lambda row: row['admission_date_time'].day == row['op_date_time'].day < row['discharge_date_time'].day, 'SDA')
]

In [820]:
df_cohort = utils.create_subgroups(
    df=df_cohort, 
    conditions=conditions
    )

In [ ]:
df_cohort

# Report (Full Cohort)

In [ ]:
cohort_report = ProfileReport(df=df_cohort, title='Cohort', minimal=True)
cohort_report.to_file('data/cohort_report.html')

# Exploratory Data Analysis (All Data)

- ```df_op``` all cases with OP
- ```df_op_cohort``` all cohort cases with OP
- ```df_cohort``` only earliest OP of cohort (main table for analysis)
- ```df_ndia``` all diagnoses
- ```df_prior_diagnoses``` all diagnoses prior to OP

In [823]:
df_cohort_eda = df_cohort.copy()

In [824]:
df_op_eda = df_op.copy()
df_op_cohort_eda = df_op_cohort.copy()
df_diagnoses_eda = df_ndia.copy()
df_prior_diagnoses_eda = df_prior_diagnoses.copy()
df_elixhauser_definition_eda = df_elixhauser_definition.copy()

## df_op

In [ ]:
df_op_eda.dtypes

In [ ]:
utils.get_eda_metrics(df=df_op_eda)

In [ ]:
df_op_eda['ops_code'].value_counts().head(10)

## df_op_cohort

In [ ]:
df_op_cohort_eda.dtypes

In [ ]:
utils.get_eda_metrics(df=df_op_cohort_eda)

In [ ]:
df_op_cohort_eda['ops_code'].value_counts().head(10)

## df_cohort

In [831]:
# view all rows
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

In [ ]:
df_cohort_eda.dtypes

In [ ]:
utils.get_eda_metrics(df=df_cohort_eda)

In [834]:
pd.reset_option('all')

### Admission Times

In [ ]:
yearly_counts = df_cohort['admission_date_time'].dt.year.value_counts().sort_index()

plt.figure(figsize=(10, 6))
plt.bar(yearly_counts.index, yearly_counts.values)
plt.xlabel('Year of Admission')
plt.ylabel('Count')
plt.title('Count of Cases per Year')
plt.xticks(yearly_counts.index) 
plt.grid(axis='y', linestyle='--', alpha=0.7)

plt.tight_layout()
plt.show()

### OP Times

In [ ]:
yearly_counts = df_cohort['op_date_time'].dt.year.value_counts().sort_index()

plt.figure(figsize=(10, 6))
plt.bar(yearly_counts.index, yearly_counts.values)
plt.xlabel('Year of OP')
plt.ylabel('Count')
plt.title('Count of Cases per Year')
plt.xticks(yearly_counts.index) 
plt.grid(axis='y', linestyle='--', alpha=0.7)

plt.tight_layout()
plt.show()

### OP Codes

In [ ]:
value_counts = df_cohort['ops_code'].value_counts()
df_value_counts = value_counts.reset_index()
df_value_counts.columns = ['ops_code', 'count']
df_value_counts

In [838]:
df_value_counts.to_csv('data/ops_codes.csv', index=False)

### Length of Stay

In [ ]:
admission_dates = pd.to_datetime(df_cohort_eda['admission_date_time'])
discharge_dates = pd.to_datetime(df_cohort_eda['discharge_date_time'])
length_of_stay = discharge_dates - admission_dates
length_of_stay = length_of_stay.fillna(pd.NaT)

df_length_of_stay = pd.DataFrame({
    'case_id': df_cohort_eda['case_id'],
    'length_of_stay': length_of_stay
})

utils.get_eda_metrics(df_length_of_stay)

### Missing Discharge

In [840]:
missing_discharge = df_cohort_eda[df_cohort_eda['discharge_date_time'].isna()]

In [841]:
missing_discharge['admission_end_of_2024'] = missing_discharge['admission_date_time'].dt.year >= 2024
missing_discharge['admission_end_of_2024'] &= missing_discharge['admission_date_time'].dt.month >= 6

In [842]:
missing_discharge = missing_discharge[['case_id', 'in_hospital_death', 'admission_end_of_2024']]

In [ ]:
print(f'Number of unique cases with missing discharge date: {missing_discharge["case_id"].nunique()}')

In [ ]:
print(f'Number of duplicate cases with missing discharge date: {missing_discharge.duplicated().sum()}')

In [ ]:
missing_discharge_death = missing_discharge[missing_discharge['in_hospital_death'] == 1]

print(f'Number of unique in hospital death cases with missing discharge date: {missing_discharge_death["case_id"].nunique()}')

In [ ]:
missing_discharge_end = missing_discharge[missing_discharge['admission_end_of_2024']]

print(f'Number of unique late 2024 admission cases with missing discharge date: {missing_discharge_end["case_id"].nunique()}')

### Elixhauser Categories

In [ ]:
elixhauser_categories = df_elixhauser_definition_eda['category'].tolist()
elixhauser_categories = [category + '_elixhauser' for category in elixhauser_categories]
elixhauser_categories = [category.lower().replace(' ', '_') for category in elixhauser_categories]
elixhauser_categories = [category.replace(',', '') for category in elixhauser_categories]
elixhauser_diagnoses_count = df_cohort_eda[df_cohort_eda[elixhauser_categories].any(axis=1)]['case_id'].nunique()
patient_count = df_cohort_eda['pat_id'].nunique()

print(f'Number of patients with (prior) Elixhauser diagnosis: {elixhauser_diagnoses_count} ({round((elixhauser_diagnoses_count / patient_count) * 100, 2) }%)')

### Outcomes

In [ ]:
stroke_count = df_cohort_eda['stroke_30_days'].sum() 
patient_count = df_cohort_eda['pat_id'].nunique()

print(f'Number of patients with stroke 30 days after OP: {stroke_count} ({round((stroke_count / patient_count) * 100, 2) }%)')

In [ ]:
MACE_count = df_cohort_eda['MACE_30_days'].sum() 
patient_count = df_cohort_eda['pat_id'].nunique()

print(f'Number of patients with MACE 30 days after OP: {MACE_count} ({round((MACE_count / patient_count) * 100, 2) }%)')

In [ ]:
death_count = df_cohort_eda['in_hospital_death'].sum() 
patient_count = df_cohort_eda['pat_id'].nunique()

print(f'Number of patients who died during hospital stay: {death_count} ({round((death_count / patient_count) * 100, 2) }%)')

### High Risk Deaths

In [851]:
df_high_risk_cohort = df_cohort_eda.copy()

#### Elixhauser

In [852]:
# get lowest value of top risk category
quantile = 0.85
elixhauser_threshold = df_high_risk_cohort['elixhauser_van_walraven'].quantile(quantile)
df_top_elixhauser_cases = df_high_risk_cohort[df_high_risk_cohort['elixhauser_van_walraven'] >= elixhauser_threshold]
min_elixhauser_threshold = df_top_elixhauser_cases['elixhauser_van_walraven'].min()

In [ ]:
# get percentage of deaths in this category
death_count_total = df_high_risk_cohort['in_hospital_death'].sum()
death_count_top_cases = df_top_elixhauser_cases['in_hospital_death'].sum()
death_percentage_top_cases = (death_count_top_cases / death_count_total)
print(f'Percentage of deaths in the top {round(1 - quantile, 2) * 100}% of patients with highest Elixhauser score: {round(death_percentage_top_cases * 100, 2)}%')

In [ ]:
print(f'Patients with high Elixhauser make up {round((len(df_top_elixhauser_cases) / len(df_high_risk_cohort)) * 100, 2)} % of cases')

#### Elevated Risk Surgery

In [ ]:
# get percentage of deaths 
death_count_total = df_high_risk_cohort['in_hospital_death'].sum()
df_elevated_risk_cases = df_high_risk_cohort[df_high_risk_cohort['elevated_risk_surgery'] == 1]
death_count_top_cases = df_elevated_risk_cases['in_hospital_death'].sum()
death_percentage_top_cases = (death_count_top_cases / death_count_total)
print(f'Percentage of deaths with elevated risk surgery patients: {round(death_percentage_top_cases * 100, 2)}%')

In [ ]:
print(f'Elevated risk patients make up {round((len(df_elevated_risk_cases) / len(df_high_risk_cohort)) * 100, 2)} % of cases')

#### Combined

In [857]:
# combine categories
df_combined_cohorts = df_high_risk_cohort[(df_high_risk_cohort['elixhauser_van_walraven'] >= elixhauser_threshold) | (df_high_risk_cohort['elevated_risk_surgery'] == 1)]

In [ ]:
# get percentage of deaths 
death_count_total = df_high_risk_cohort['in_hospital_death'].sum()
death_count_top_cases = df_combined_cohorts['in_hospital_death'].sum()
death_percentage_top_cases = (death_count_top_cases / death_count_total)
print(f'Percentage of deaths with either high Elixhauser or elevated risk surgery: {round(death_percentage_top_cases * 100, 2)}%')

In [ ]:
print(f'High risk patients make up {round((len(df_combined_cohorts) / len(df_high_risk_cohort)) * 100, 2)} % of cases')

### OP Times

In [860]:
df_cohort_eda_times = df_cohort_eda.copy()

In [861]:
# round to hour
df_cohort_eda_times['hour_rounded'] = df_cohort_eda_times['op_date_time'].dt.round('H').dt.hour

In [862]:
# get frequencies
hour_counts = df_cohort_eda_times['hour_rounded'].value_counts().sort_index()

In [ ]:
plt.figure(figsize=(10,6))
hour_counts.plot(kind='bar')
plt.title('Frequency of Rounded Hours')
plt.xlabel('Hour of the Day')
plt.ylabel('Frequency')
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()

### Outcomes / Score

In [864]:
score_columns = ['RCRI_original', 'RCRI_recalibrated']
outcome_columns = ['MACE_30_days', 'MACE_30_days']

In [ ]:
for score, outcome in zip(score_columns, outcome_columns):
    result = (df_cohort_eda.groupby(score)[outcome].sum().reset_index())
    result.columns = [score, f'sum_of_{outcome}']
    print(result)

## df_diagnoses

In [ ]:
df_diagnoses_eda.dtypes

In [ ]:
utils.get_eda_metrics(df=df_diagnoses_eda)

In [ ]:
df_diagnoses_eda['icd_code'].value_counts().head(10)

In [ ]:
unique_diagnosed_patients = df_diagnoses_eda['case_id'].drop_duplicates()
diagnoses_count = unique_diagnosed_patients.isin(df_cohort_eda['case_id']).sum()
# diagnoses_count = df_cohort_eda['case_id'].isin(df_diagnoses_eda['case_id']).astype(int).sum()
patient_count = df_cohort_eda['pat_id'].nunique()

print(f'Number of patients with diagnosis: {diagnoses_count} ({round((diagnoses_count / patient_count) * 100, 2)}%)') 

In [ ]:
case_diagnoses_count = df_diagnoses_eda.groupby('case_id').size()
median_case_diagnoses = case_diagnoses_count.median()
print(f'Median number of diagnoses per case: {round(median_case_diagnoses, 4)}') 

## df_prior_diagnoses

In [ ]:
df_prior_diagnoses_eda.dtypes

In [ ]:
utils.get_eda_metrics(df=df_prior_diagnoses_eda)

In [ ]:
df_prior_diagnoses_eda['icd_code'].value_counts().head(10)

In [ ]:
unique_prior_diagnosed_patients = df_prior_diagnoses_eda['case_id'].drop_duplicates()
prior_diagnoses_count = df_cohort_eda['case_id'].isin(unique_prior_diagnosed_patients).astype(int).sum()
patient_count = df_cohort_eda['pat_id'].nunique()

print(f'Number of patients with prior diagnosis: {prior_diagnoses_count} ({round((prior_diagnoses_count / patient_count) * 100, 2)}%)') 

In [ ]:
case_prior_diagnoses_count = df_prior_diagnoses_eda.groupby('case_id').size()
median_case_prior_diagnoses = case_prior_diagnoses_count.median()
print(f'Median number of prior diagnoses per case: {round(median_case_prior_diagnoses, 4)}') 

# Final Cleaning

In [ ]:
# remove columns with missings
threshold = len(df_cohort) * 0.75
df_cohort_cleaned = df_cohort.copy().dropna(thresh=threshold, axis=1)
removed_columns = set(df_cohort.columns) - set(df_cohort_cleaned.columns)
print(f'Number of removed columns: {len(removed_columns)}')
print(f'Columns that  got removed: {removed_columns}')

In [ ]:
df_cohort_cleaned

In [ ]:
df_cohort_validation = df_cohort_cleaned.dropna(how='any')
utils.get_amount_removed_rows(
    initial=df_cohort_cleaned, 
    new=df_cohort_validation
    )

In [ ]:
df_cohort_validation

# Conversion of Scores

In [881]:
original_score_columns = [
    'RCRI_original', 'RCRI_original', 
    'CHA2DS2_VASc_original', 'CHA2DS2_VASc_original', 
    'elixhauser_van_walraven', 'elixhauser_van_walraven'
    ]

new_score_columns = [
    'RCRI_recalibrated', 'expanded_RCRI', 
    'CHA2DS2_VASc_recalibrated', 'expanded_CHA2DS2_VASc', 
    'elixhauser_recalibrated', 'expanded_elixhauser'
    ]

In [ ]:
for original_score, new_score in zip(original_score_columns, new_score_columns):
    print(f'Converting: {new_score}')
    df_cohort_validation = analysis_utils.convert_score(
        df=df_cohort_validation, 
        original_score=original_score, 
        new_score=new_score
        )

# Outcome Probabilities / Score

In [883]:
score_columns = [
    'RCRI_original', 'RCRI_recalibrated_converted', 'expanded_RCRI_converted', 
    'CHA2DS2_VASc_original', 'CHA2DS2_VASc_recalibrated_converted', 'expanded_CHA2DS2_VASc_converted', 
    'elixhauser_van_walraven', 'elixhauser_recalibrated_converted', 'expanded_elixhauser_converted'
    ]

outcome_columns = [
    'MACE_30_days', 'MACE_30_days', 'MACE_30_days', 
    'stroke_30_days', 'stroke_30_days', 'stroke_30_days', 
    'in_hospital_death', 'in_hospital_death', 'in_hospital_death'
    ]

categorical_columns = [
    'RCRI_original', 'RCRI_recalibrated_converted', 'expanded_RCRI_converted', 
    'CHA2DS2_VASc_original', 'CHA2DS2_VASc_recalibrated_converted', 'expanded_CHA2DS2_VASc_converted', 
    ]

In [ ]:
for score, outcome in zip(score_columns, outcome_columns):
    print(f'Calculating probabilities for: {score}')
    categorical_column = []
    if score in categorical_columns:
        categorical_column = [score]

    analysis_utils.get_probabilities_for_cohort(
        df=df_cohort_validation, 
        score_column=score, 
        outcome_column=outcome, 
        test_size=train_test_split, 
        categorical_columns=categorical_column
        )
    
    analysis_utils.get_confidence_intervals(
        df=df_cohort_validation, 
        score_column=score, 
        outcome_column=outcome, 
        test_size=train_test_split, 
        categorical_columns=categorical_column
        )

In [ ]:
df_cohort_validation

# Exploratory Data Analysis (Analysis Cohort)

In [886]:
# view all rows
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

## Raw Tables

In [ ]:
utils.get_eda_metrics(df=df_hdl_copra_hierarchy)

In [ ]:
utils.get_eda_metrics(df=df_hdl_sap_procedure)

In [ ]:
utils.get_eda_metrics(df=df_hdl_sap_patient)

In [ ]:
utils.get_eda_metrics(df=df_sap_lab)

In [ ]:
utils.get_eda_metrics(df=df_hdl_sap_fall)

In [ ]:
utils.get_eda_metrics(df=df_hdl_copra_diagnose)

In [ ]:
utils.get_eda_metrics(df=df_hdl_copra_medication)

In [ ]:
utils.get_eda_metrics(df=df_sap_movement)

## Processed Raw Tables

In [ ]:
utils.get_eda_metrics(df=df_hierarchy)

In [ ]:
utils.get_eda_metrics(df=df_nicp)

In [ ]:
utils.get_eda_metrics(df=df_npat)

In [ ]:
utils.get_eda_metrics(df=df_lab)

In [ ]:
utils.get_eda_metrics(df=df_cohort_complete)

In [ ]:
utils.get_eda_metrics(df=df_medication)

In [ ]:
utils.get_eda_metrics(df=df_campi)

## Derived Tables

In [ ]:
utils.get_eda_metrics(df=df_cohort)

In [ ]:
utils.get_eda_metrics(df=df_cohort_validation)

In [ ]:
utils.get_eda_metrics(df=df_prior_diagnoses)

In [ ]:
utils.get_eda_metrics(df=df_prior_medication)

In [ ]:
utils.get_eda_metrics(df=df_prior_lab)

In [ ]:
utils.get_eda_metrics(df=df_prior_lab)

In [ ]:
subgroups = utils.collect_subgroups(
    df=df_cohort_validation, 
    conditions=conditions
    )
subgroups.keys()

In [ ]:
utils.get_eda_metrics(df=subgroups['female'])

In [ ]:
utils.get_eda_metrics(df=subgroups['male'])

In [ ]:
utils.get_eda_metrics(df=subgroups['age_above_65'])

In [ ]:
utils.get_eda_metrics(df=subgroups['age_below_65'])

In [ ]:
utils.get_eda_metrics(df=subgroups['asa_le_2'])

In [ ]:
utils.get_eda_metrics(df=subgroups['asa_gt_2'])

In [ ]:
utils.get_eda_metrics(df=subgroups['ambulatory'])

In [ ]:
utils.get_eda_metrics(df=subgroups['inpatient'])

In [ ]:
utils.get_eda_metrics(df=subgroups['SDA'])

In [ ]:
utils.get_eda_metrics(df=subgroups['campus_mitte'])

In [ ]:
utils.get_eda_metrics(df=subgroups['campus_steglitz'])

In [ ]:
utils.get_eda_metrics(df=subgroups['campus_wedding'])

In [ ]:
utils.get_eda_metrics(
    df=analysis_utils.get_train_data(
        df=df_cohort_validation, 
        test_size=train_test_split
        )
    )

In [ ]:
utils.get_eda_metrics(
    df=analysis_utils.get_test_data(
        df=df_cohort_validation, 
        test_size=train_test_split
        )
    )

In [925]:
pd.reset_option('all')

# Report & Export (Analysis Cohort)

In [ ]:
cleaned_cohort_report = ProfileReport(df=df_cohort_validation, title='Cleaned Cohort', minimal=True)
cleaned_cohort_report.to_file('data/cleaned-cohort_report.html')

In [927]:
df_cohort_validation.to_csv(path_or_buf='data/base/cleaned_cohort_data.csv', index=False)

# Predictors

A list of all predictor columns, mainly used for regression and machine learning models later on.

In [14]:
# Existing variables
data_vars = list(df_cohort_validation.columns)
columns_to_remove = [
    'pat_id', 'case_id', 'admission_date_time', 'discharge_date_time', 'ops_code', 'op_date_time', 'birth_date', 
    'in_hospital_death', 'MACE_30_days', 'stroke_30_days',
    'elixhauser_van_walraven', 'elixhauser_recalibrated', 'expanded_elixhauser', 
    'elixhauser_recalibrated_converted', 'expanded_elixhauser_converted', 
    'RCRI_original', 'RCRI_recalibrated', 'expanded_RCRI', 
    'RCRI_recalibrated_converted', 'expanded_RCRI_converted', 
    'CHA2DS2_VASc_original', 'CHA2DS2_VASc_recalibrated', 'expanded_CHA2DS2_VASc', 
    'CHA2DS2_VASc_recalibrated_converted', 'expanded_CHA2DS2_VASc_converted',

    'RCRI_original_probability',
    'RCRI_original_probability_CI_lower',
    'RCRI_original_probability_CI_upper',
    'RCRI_recalibrated_converted_probability',
    'RCRI_recalibrated_converted_probability_CI_lower',
    'RCRI_recalibrated_converted_probability_CI_upper',
    'expanded_RCRI_converted_probability',
    'expanded_RCRI_converted_probability_CI_lower',
    'expanded_RCRI_converted_probability_CI_upper',
    'CHA2DS2_VASc_original_probability',
    'CHA2DS2_VASc_original_probability_CI_lower',
    'CHA2DS2_VASc_original_probability_CI_upper',
    'CHA2DS2_VASc_recalibrated_converted_probability',
    'CHA2DS2_VASc_recalibrated_converted_probability_CI_lower',
    'CHA2DS2_VASc_recalibrated_converted_probability_CI_upper',
    'expanded_CHA2DS2_VASc_converted_probability',
    'expanded_CHA2DS2_VASc_converted_probability_CI_lower',
    'expanded_CHA2DS2_VASc_converted_probability_CI_upper',
    'elixhauser_van_walraven_probability',
    'elixhauser_van_walraven_probability_CI_lower',
    'elixhauser_van_walraven_probability_CI_upper',
    'elixhauser_recalibrated_converted_probability',
    'elixhauser_recalibrated_converted_probability_CI_lower',
    'elixhauser_recalibrated_converted_probability_CI_upper',
    'expanded_elixhauser_converted_probability',
    'expanded_elixhauser_converted_probability_CI_lower',
    'expanded_elixhauser_converted_probability_CI_upper',

    'female',
    'male',
    'campus'
]

data_vars = [col for col in data_vars if col not in columns_to_remove]

In [ ]:
data_vars

In [16]:
CHA_variables = [
    'female_sex', 'congestive_heart_failure_elixhauser', 'hypertension_uncomplicated_elixhauser', 
    'hypertension_complicated_elixhauser', 'diabetes_uncomplicated_elixhauser', 'diabetes_complicated_elixhauser', 
    'vascular_disease_history', 'STT_history', 'age_below_65', 'age_between_65_and_74', 'age_above_74']

RCRI_variables = [
    'elevated_risk_surgery', 'MI_history', 'congestive_heart_failure_elixhauser', 
    'CD_history', 'prior_insulin', 'prior_creatinine'
    ]

elix_variables = [col for col in df_cohort_validation.columns if col.endswith('_elixhauser')]

In [ ]:
CHA_variables

In [ ]:
RCRI_variables

In [ ]:
elix_variables

# Outcomes

In [20]:
stroke_outcome = 'stroke_30_days'
MACE_outcome = 'MACE_30_days'
death_outcome = 'in_hospital_death'

# Automated Mapping

In [ ]:
# get prior diagnoses and outcomes
df_automated_mapping = pd.merge(
    df_cohort_validation[['case_id', 'MACE_30_days', 'stroke_30_days', 'in_hospital_death']], df_prior_diagnoses[['case_id', 'icd_code']], 
    on='case_id', how='inner')

In [ ]:
print(f'Amount of cases: {len(df_automated_mapping)}')

In [ ]:
df_automated_mapping['icd_code'].value_counts()

In [938]:
# clean data to only keep broader category
df_automated_mapping['icd_code'] = df_automated_mapping['icd_code'].apply(lambda x: x[:3])

In [939]:
# keep only those that at least 1% of cases have
icd_code_counts = df_automated_mapping['icd_code'].value_counts() # ICDs counts
threshold = 0.01 * len(df_automated_mapping['case_id'].unique()) # threshold count
icds_to_keep = icd_code_counts[icd_code_counts >= threshold].index # selected ICDs
df_automated_mapping = df_automated_mapping[df_automated_mapping['icd_code'].isin(icds_to_keep)]

In [ ]:
print(f'Amount of cases: {len(df_automated_mapping)}')

In [ ]:
df_automated_mapping['icd_code'].value_counts()

In [942]:
df_automated_mapping = analysis_utils.get_train_data(
    df=df_automated_mapping, 
    test_size=train_test_split, 
    calculate_proba=False
    )

## MACE

In [ ]:
# get important features
selected_MACE_features = analysis_utils.get_important_features(
    df=df_automated_mapping, 
    feature_column='icd_code', 
    outcome_column='MACE_30_days', 
    n_features=15
    )

In [944]:
# generate column names
selected_MACE_features_columns = []
df_cohort_expanded_MACE = df_cohort_validation.copy()
for feature in selected_MACE_features:
    column_name = f'{feature}_ICD_history'
    selected_MACE_features_columns.append(column_name)

    icd_pattern = f'^{feature}'
    df_feature = utils.extract_df_data(
        df=df_prior_diagnoses, 
        filter_dict={'icd_code':[icd_pattern]}
        )
    
    df_cohort_expanded_MACE[column_name] = df_cohort_expanded_MACE['case_id'].isin(df_feature['case_id']).astype(int)

## CHA2DS2-VASc

In [ ]:
# get important features
selected_CHA_features = analysis_utils.get_important_features(
    df=df_automated_mapping, 
    feature_column='icd_code', 
    outcome_column='stroke_30_days', 
    n_features=15
    )

In [946]:
# generate column names
selected_CHA_features_columns = []
df_cohort_expanded_CHA = df_cohort_validation.copy()
for feature in selected_CHA_features:
    column_name = f'{feature}_ICD_history'
    selected_CHA_features_columns.append(column_name)

    icd_pattern = f'^{feature}'
    df_feature = utils.extract_df_data(
        df=df_prior_diagnoses, 
        filter_dict={'icd_code':[icd_pattern]}
        )
    
    df_cohort_expanded_CHA[column_name] = df_cohort_expanded_CHA['case_id'].isin(df_feature['case_id']).astype(int)

## Elixhauser

In [ ]:
# get important features
selected_elix_features = analysis_utils.get_important_features(
    df=df_automated_mapping, 
    feature_column='icd_code', 
    outcome_column='in_hospital_death', 
    n_features=15
    )

In [948]:
# generate column names
selected_elix_features_columns = []
df_cohort_expanded_elix = df_cohort_validation.copy()
for feature in selected_elix_features:
    column_name = f'{feature}_ICD_history'
    selected_elix_features_columns.append(column_name)

    icd_pattern = f'^{feature}'
    df_feature = utils.extract_df_data(
        df=df_prior_diagnoses, 
        filter_dict={'icd_code':[icd_pattern]}
        )
    
    df_cohort_expanded_elix[column_name] = df_cohort_expanded_elix['case_id'].isin(df_feature['case_id']).astype(int)

# Recalibration

## RCRI

In [ ]:
X = df_cohort_validation[RCRI_variables].copy()
y = df_cohort_validation[MACE_outcome].copy()

df_RCRI_weights = analysis_utils.get_regression(
    X=X, 
    y=y, 
    label_name='RCRI', 
    test_size=train_test_split, 
    scale_data=False, 
    use_lasso=False
    )

In [ ]:
df_RCRI_weights_normalized = analysis_utils.normalize_weights(
    df=df_RCRI_weights, 
    save_to='data/RCRI-logit-weights.csv'
    )
df_RCRI_weights_normalized

## Expanded RCRI

In [ ]:
X = df_cohort_expanded_MACE[RCRI_variables + selected_MACE_features_columns].copy()
y = df_cohort_expanded_MACE[MACE_outcome].copy()

df_expanded_RCRI_weights = analysis_utils.get_regression(
    X=X, 
    y=y, 
    label_name='expanded RCRI', 
    test_size=train_test_split, 
    scale_data=False, 
    use_lasso=False
    )

In [ ]:
df_expanded_RCRI_weights_normalized = analysis_utils.normalize_weights(
    df=df_expanded_RCRI_weights, 
    save_to='data/expanded-RCRI-logit-weights.csv'
    )
df_expanded_RCRI_weights_normalized

## CHA2DS2-VASc

In [ ]:
X = df_cohort_validation[CHA_variables].copy()
y = df_cohort_validation[stroke_outcome].copy()

df_CHA_weights = analysis_utils.get_regression(
    X=X, 
    y=y, 
    label_name='CHA2DS2-VASc', 
    test_size=train_test_split, 
    scale_data=False, 
    use_lasso=False
    )
df_CHA_weights

In [ ]:
df_CHA_weights_normalized = analysis_utils.normalize_weights(
    df=df_CHA_weights, 
    save_to='data/CHA-logit-weights.csv'
    )
df_CHA_weights_normalized

## Expanded CHA2DS2-VASc

In [ ]:
X = df_cohort_expanded_CHA[CHA_variables + selected_CHA_features_columns].copy()
y = df_cohort_expanded_CHA[stroke_outcome].copy()

df_expanded_CHA_weights = analysis_utils.get_regression(
    X=X, 
    y=y, 
    label_name='Expanded CHA2DS2-VASc', 
    test_size=train_test_split, 
    scale_data=False, 
    use_lasso=False
    )
df_expanded_CHA_weights

In [ ]:
df_expanded_CHA_weights_normalized = analysis_utils.normalize_weights(
    df=df_expanded_CHA_weights, 
    save_to='data/expanded-CHA-logit-weights.csv'
    )
df_expanded_CHA_weights_normalized

## Elixhauser

In [ ]:
X = df_cohort_validation[elix_variables].copy()
y = df_cohort_validation[death_outcome].copy()

df_elix_weights = analysis_utils.get_regression(
    X=X, 
    y=y, 
    label_name='Elixhauser', 
    test_size=train_test_split, 
    scale_data=False, 
    use_lasso=False
    )
df_elix_weights

In [ ]:
df_elix_weights_normalized = analysis_utils.normalize_weights(
    df=df_elix_weights, 
    save_to='data/elixhauser-logit-weights.csv'
    )
df_elix_weights_normalized

## Expanded Elixhauser

In [ ]:
selected_elix_features_columns

In [ ]:
X = df_cohort_expanded_elix[elix_variables + selected_elix_features_columns].copy()
y = df_cohort_expanded_elix[death_outcome].copy()

df_expanded_elix_weights = analysis_utils.get_regression(
    X=X, 
    y=y, 
    label_name='Expanded Elixhauser', 
    test_size=train_test_split, 
    scale_data=False, 
    use_lasso=False
    )
df_expanded_elix_weights

In [ ]:
df_expanded_elix_weights_normalized = analysis_utils.normalize_weights(
    df=df_expanded_elix_weights, 
    save_to='data/expanded-elixhauser-logit-weights.csv'
    )
df_expanded_elix_weights_normalized

# Machine Learning [WIP]

## Custom

In [960]:
# get data
variables = RCRI_variables + list(df_expanded_RCRI_weights[df_expanded_RCRI_weights['feature'] != 'const']['feature'])
outcome = MACE_outcome

In [ ]:
# prepare data
X_train, X_test, y_train, y_test = ml_utils.preprocessing(
    df=df_cohort_expanded_MACE, 
    variables=variables, 
    outcome=outcome, 
    scale=True, 
    resample=True, 
    test_size=train_test_split
    )

In [962]:
# inspect data
ml_utils.get_eda_metrics(
    X_train=X_train, 
    X_test=X_test, 
    y_train=y_train, 
    y_test=y_test, 
    variables=variables, 
    outcome=outcome
    )

In [ ]:
# prepare, train, and evaluate model
 
class_weights = ml_utils.get_class_weights(y_train=y_train)

model, criterion, optimizer = ml_utils.get_model(
    features=len(variables), 
    class_weights_tensor=class_weights, 
    selected_model=ml_utils.Model.CUSTOM
    )

ml_utils.train(
    X_train=X_train, 
    y_train=y_train, 
    model=model, 
    criterion=criterion, 
    optimizer=optimizer, 
    epochs=800
    )

ml_utils.evaluate(
    model=model, 
    X_train=X_train, 
    y_train=y_train, 
    X_test=X_test, 
    y_test=y_test
    )

In [964]:
# save model
ml_utils.export_model_state(
    model=model, 
    path='data/ml/', 
    name='custom-model_v1'
    )

## AutoML

In [965]:
# get data
data = df_cohort_expanded_MACE[
    RCRI_variables + list(df_expanded_RCRI_weights[df_expanded_RCRI_weights['feature'] != 'const']['feature']) + [MACE_outcome]
    ].copy() 

In [966]:
# build and test models
autoML_model = ml_utils.get_autoML(
    data=data, 
    target=MACE_outcome
    )

# Validation

In [27]:
train_test_split = train_test_split

In [ ]:
subgroups = utils.collect_subgroups(
    df=df_cohort_validation, 
    conditions=conditions
    )
subgroups.keys()

## RCRI

In [ ]:
analysis_utils.validate_score(
    df=df_cohort_validation, 
    score_columns=['RCRI_original', 'RCRI_recalibrated_converted', 'expanded_RCRI_converted'], 
    outcome_column='MACE_30_days', 
    test_size=train_test_split, 
    dca_y_limits=[-0.002, 0.010], 
    categorical_columns=['RCRI_original', 'RCRI_recalibrated', 'expanded_RCRI']
    )

### Subgroups

In [ ]:
analysis_utils.evaluate_subgroups(
    subgroups=subgroups, 
    score_columns=['RCRI_original', 'RCRI_recalibrated_converted', 'expanded_RCRI_converted'], 
    outcome_column='MACE_30_days', 
    test_size=train_test_split, 
    categorical_columns=['RCRI_original', 'RCRI_recalibrated_converted', 'expanded_RCRI_converted']
    )

## CHA2DS2-VASc

In [971]:
# cohort for CHA2DS2-VASc
df_cohort_validation_AF = df_cohort_validation[df_cohort_validation['AF_history'] == 1]

In [ ]:
analysis_utils.validate_score(
    df=df_cohort_validation, 
    score_columns=['CHA2DS2_VASc_original', 'CHA2DS2_VASc_recalibrated_converted', 'expanded_CHA2DS2_VASc_converted'], 
    outcome_column='stroke_30_days', 
    test_size=train_test_split, 
    dca_y_limits=[-0.002, 0.010], 
    categorical_columns=['CHA2DS2_VASc_original', 'CHA2DS2_VASc_recalibrated_converted', 'expanded_CHA2DS2_VASc_converted']
    )

### Subgroups

In [ ]:
analysis_utils.evaluate_subgroups(
    subgroups=subgroups, 
    score_columns=['CHA2DS2_VASc_original', 'CHA2DS2_VASc_recalibrated_converted', 'expanded_CHA2DS2_VASc_converted'], 
    outcome_column='stroke_30_days', 
    test_size=train_test_split, 
    categorical_columns=['CHA2DS2_VASc_original', 'CHA2DS2_VASc_recalibrated_converted', 'expanded_CHA2DS2_VASc_converted']
    )

## Elixhauser

In [ ]:
analysis_utils.validate_score(
    df=df_cohort_validation, 
    score_columns=['elixhauser_van_walraven', 'elixhauser_recalibrated_converted', 'expanded_elixhauser_converted'], 
    outcome_column='in_hospital_death', 
    test_size=train_test_split, 
    dca_y_limits=[-0.002, 0.002], 
    dca_thresholds=np.arange(0, 0.05, 0.01), 
    categorical_columns=['elixhauser_van_walraven', 'elixhauser_recalibrated_converted', 'expanded_elixhauser_converted']
    )

### Subgroups

In [ ]:
analysis_utils.evaluate_subgroups(
    subgroups=subgroups, 
    score_columns=['elixhauser_van_walraven', 'elixhauser_recalibrated_converted', 'expanded_elixhauser_converted'], 
    outcome_column='in_hospital_death', 
    test_size=train_test_split, 
    categorical_columns=['elixhauser_van_walraven', 'elixhauser_recalibrated_converted', 'expanded_elixhauser_converted']
    )

# Other

## Sample

In [976]:
df_cohort_sample = df_cohort.copy()

In [977]:
selection_conditions = [
    (lambda row: row['campus'] == 'S', 'sample_base'),  # campus
    (lambda row: pd.isna(row['asa_status']), 'sample_criteria_a'), # missings
    (lambda row: (row['AF_history'] == 1), 'sample_criteria_b'), # prior
    (lambda row: (row['stroke_30_days'] == 1), 'sample_criteria_c'), # post
]

In [978]:
df_cohort_sample = utils.create_subgroups(
    df=df_cohort_sample, 
    conditions=selection_conditions
    )

In [979]:
# get all from campus
df_base = df_cohort_sample[df_cohort_sample['sample_base'] == 1]

In [980]:
# get all with sample criteria
df_a = df_base[df_base['sample_criteria_a'] == 1]
df_b = df_base[df_base['sample_criteria_b'] == 1]
df_c = df_base[df_base['sample_criteria_c'] == 1]

In [981]:
# randomly select with sample criteria
sample_a = df_a.sample(n=5, random_state=42)
sample_b = df_b.sample(n=5, random_state=42)
sample_c = df_c.sample(n=5, random_state=42)

In [982]:
# randomly select without sample criteria
remaining_cases = df_base.drop(sample_a.index).drop(sample_b.index).drop(sample_c.index).sample(n=5, random_state=42)

In [983]:
# group all
df_selected = pd.concat([sample_a, sample_b, sample_c, remaining_cases])

In [ ]:
# inspect
utils.get_eda_metrics(df=df_selected)

In [985]:
# export
df_selected.to_csv(path_or_buf='data/cohort_data_sample.csv', index=False)

In [ ]:
# build report
cohort_sample_report = ProfileReport(df=df_selected, title='Cohort Sample', minimal=True)
cohort_sample_report.to_file('data/cohort_samle_report.html')